In [ ]:
import os
from set_home_directory import get_project_root_homedir_in_sys_path
project_root, main_dir = get_project_root_homedir_in_sys_path("inter_areal_predictability")
if project_root is None:
    raise RuntimeError(f"Project root not found: ensure a folder named '{project_root}' exists in one of the sys.path entries.")
print("Project root found:", project_root)
os.chdir(project_root)
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import json
import random
from matplotlib import gridspec
from matplotlib.patches import Patch
from sklearn.mixture import GaussianMixture
from matplotlib import gridspec


fig_dir = project_root + '/results/paper_figures/'
save_figs = False
import sys
sys.path.insert(0,os.path.join(main_dir,'utils/'))
sys.path.insert(0,main_dir)


from utils.neuron_properties_functions import create_empty_mouse_stats_dict, get_split_half_r_all_mice, get_SNR_all_mice, get_max_corr_vals_all_mice, get_evars_all_mice
from utils.fig_6_functions import get_norm_variance_all_mice, get_1_vs_rest_all_mice, make_mouse_df_neuron_properties, make_corr_df_mouse, set_corr_figs_params, add_stars_5_sets,plot_mouse, add_ax_properties_corr, plot_spont_figs
from utils.neuron_properties_functions import create_empty_monkey_stats_dict, get_SNR_monkey_all_dates, get_split_half_r_monkey_all_dates,get_max_corr_vals_monkey_all_dates,get_evar_monkey_all_dates, store_macaque_alphas
from utils.fig_6_functions import make_monkey_df_neuron_properties,make_corr_df_monkey, plot_corr_bars, add_group_pairwise_stars_1_set, plot_bars, plot_date, get_one_vs_rest_r_monkey_all_dates, get_electrode_ids_all_dates
from utils.fig_6_functions import get_predictor_indices_elec_ids,get_xtarget_predictor_indices_elecs,get_x_target_overlap_evars
from utils.fig_6_functions import plot_cell_rf_overlaps, plot_rf_overlap_performance, create_df_overlaps, pairwise_within_x_using_your_tests, p_to_stars
from utils.macaque_data_functions import get_get_condition_type
from utils.stats_functions import get_comparison_test_stars




### import stats

In [ ]:
from matplotlib.colors import LinearSegmentedColormap, Normalize
colors = ['#ddf2f0', '#55C0B7', '#112624']
norm = Normalize(vmin=0, vmax=1)
v4_cmap = LinearSegmentedColormap.from_list('custom_colormap', colors, N=256)
colors = ['#FFE8E8', '#F58B8E', '#551312']
v1_cmap = LinearSegmentedColormap.from_list('custom_colormap', colors, N=256)

In [ ]:
mouse_stats_path = os.path.join(project_root, 'results/fig_6',f'mouse_stats.pkl')
with open(mouse_stats_path, 'rb') as f:
	mouse_stats = pickle.load(f)


monkey_names = ['L','A','D']
all_monkey_stats = {}
for monkey_name in monkey_names:
    monkey_stats_path = os.path.join(project_root, 'results/fig_6',f'monkey_{monkey_name}_stats.pkl')
    with open(monkey_stats_path, 'rb') as f:
        all_monkey_stats[monkey_name] = pickle.load(f)

## Mouse neuron properties

### plotting

In [ ]:
df_mouse_ = make_mouse_df_neuron_properties(mouse_stats , dataset_types=['ori32','natimg32','ori32_spont','natimg32_spont'])
df_mouse=df_mouse_[df_mouse_['control_shuffle']==False]
df_mouse_corr = make_corr_df_mouse(mouse_stats, df_mouse)

In [ ]:
# Fig 6A left
x='Value_Type'
y='corr'
hue='Activity Type'

figsize=(1.5,1.5)
order= df_mouse_corr.Value_Type.unique()[1:]
filter_bool = df_mouse_corr['relationship'].isin(['EV_EV'])
area_bool = df_mouse_corr['Area']=='L2/3'
mask = (~filter_bool)&area_bool
palette=['#72BEB7','#B6E3DF']
hue_order= ['stimulus','gray screen']
xtick_labels = ['max r² val', '1-vs-rest r²','SNR', r'$\text{var}_{\text{stim}}$', r'$\text{var}_{\text{repeats}}$','split-half r', ]

fig, ax = plt.subplots(figsize=figsize)

df_mouse_corr_area = df_mouse_corr[mask]
sns.barplot(data=df_mouse_corr_area, x=x, palette=palette,
            y=y, order=order, hue=hue, hue_order=hue_order,
            errorbar='se', err_kws={'linewidth': 0.75})
set_corr_figs_params(ax, fontsize=6, xticklabels=xtick_labels)
legend = ax.legend(loc=(0.52,0.8), fontsize=6*0.7,labelspacing=0.3, handletextpad=0.5)
legend.get_frame().set_linewidth(0.2)
ax.tick_params(axis='x', pad=0)
ax.set_ylabel(ax.get_ylabel(), labelpad=-3)

# compute y for stars a bit above the taller of the pair
ymins, ymaxs = ax.get_ylim()
yrange = ymaxs - ymins
pvals = {}
for x_i in order:
	pvals[x_i] = get_comparison_test_stars(df_mouse_corr_area[df_mouse_corr_area[x]==x_i], independent_variable=hue, neuron_property=y, print_pval=True,
												mouse_or_date='Mouse Name',perm_type='ind')
for i, act in enumerate(order):
	if act not in pvals:
		continue
	stars = pvals[act]
	# find max y at this category from plotted data to place stars above
	sub = df_mouse_corr_area[df_mouse_corr_area[x]==act][y].to_numpy()
	sub = sub[np.isfinite(sub)]
	y_top = (np.percentile(sub, 70) if sub.size else ymaxs) + 0.03*yrange
	color = "gray" if stars=="n.s." else "black"
	ax.text(i, y_top, stars, ha="center", va="bottom", fontsize=6, color=color)

if save_figs is True:
    plt.savefig(fig_dir +'fig6_mouse_EV_corrs_resp_grayscreen_L23.pdf',transparent=True,bbox_inches='tight')


In [ ]:
# Fig 6A right
area_bool = df_mouse_corr['Area']=='L4'
mask = (~filter_bool)&area_bool
palette=['#EDAEAE', '#f6d6d6']
fig, ax = plt.subplots(figsize=figsize)
sns.barplot(data=df_mouse_corr[mask], x='Value_Type', palette=palette,
            y='corr', order=order, hue='Activity Type', hue_order=hue_order,
             errorbar='se', err_kws={'linewidth': 1})
set_corr_figs_params(ax, fontsize=6, xticklabels=xtick_labels)
sns.despine()
legend = ax.legend(loc=(0.52,0.8), fontsize=6*0.7,labelspacing=0.3, handletextpad=0.5)
legend.get_frame().set_linewidth(0.2)
ax.set_ylabel('')
ax.set_yticklabels('')


# compute y for stars a bit above the taller of the pair
ymins, ymaxs = ax.get_ylim()
yrange = ymaxs - ymins
pvals = {}
for x_i in order:
	pvals[x_i] = get_comparison_test_stars(df_mouse_corr_area[df_mouse_corr_area[x]==x_i], independent_variable=hue, neuron_property=y, print_pval=True,
												mouse_or_date='Mouse Name',perm_type='ind')
for i, act in enumerate(order):
	if act not in pvals:
		continue
	stars = pvals[act]
	# find max y at this category from plotted data to place stars above
	sub = df_mouse_corr_area[df_mouse_corr_area[x]==act][y].to_numpy()
	sub = sub[np.isfinite(sub)]
	y_top = (np.percentile(sub, 70) if sub.size else ymaxs) + 0.03*yrange
	if act =='Variance across stimuli':
		y_top = y_top + 0.08*yrange
		# print('adjusting y_top for variance across stimuli')
	color = "gray" if stars=="n.s." else "black"
	ax.text(i, y_top, stars, ha="center", va="bottom", fontsize=6, color=color)

if save_figs is True:
    plt.savefig(fig_dir +'fig6_mouse_EV_corrs_resp_grayscreen_L4.pdf',transparent=True,bbox_inches='tight')


In [ ]:

df_mouse_both=pd.merge(df_mouse[df_mouse['Activity Type']=='stimulus'],df_mouse[df_mouse['Activity Type']=='gray screen'].rename(columns={'EV':'EV gray screen','max r² val':'max r² val gray screen'}), on=['Mouse', 'Mouse Name', 'Area',
       'Direction', 'SNR', 'split-half r', '1-vs-rest r²',
       'variance', 'control_shuffle','Neuron'])

df_mouse_removed_pcs_= make_mouse_df_neuron_properties(mouse_stats , dataset_types=['ori32','natimg32','ori32_spont','natimg32_spont'], remove_pcs=True)
df_mouse_removed_pcs=df_mouse_removed_pcs_[df_mouse_removed_pcs_['control_shuffle']==False].reset_index(drop=True)
df_mouse_removed_pcs_corr = make_corr_df_mouse(mouse_stats, df_mouse_removed_pcs)
df_mouse_removed_pcs_both=pd.merge(df_mouse_removed_pcs[df_mouse_removed_pcs['Activity Type']=='stimulus'],df_mouse_removed_pcs[df_mouse_removed_pcs['Activity Type']=='gray screen'].rename(columns={'EV':'EV gray screen','max r² val':'max r² val gray screen'}), on=['Mouse', 'Mouse Name', 'Area',
       'Direction', 'SNR', 'split-half r', '1-vs-rest r²',
       'variance', 'control_shuffle','Neuron'])

In [ ]:
# Fig 6B top
neuron_properties = ['max r² val', 'split-half r', '1-vs-rest r²', ]
area='L2/3'
area_bool = df_mouse_corr['Area']==area
mask = (~filter_bool)&area_bool

df_filtered_mouse=df_mouse_both.sample(4000, random_state=17)
df_filtered_mouse_removed=df_mouse_removed_pcs_both.sample(4000, random_state=17)
df_filtered_mouse_sorted = df_filtered_mouse.sort_values('EV gray screen')



# Calculate global min and max for consistent normalization
global_min = df_filtered_mouse_sorted['EV gray screen'].min()
global_max = df_filtered_mouse_sorted['EV gray screen'].max()


figsize=(2.5,0.75)
hspace=0.3
wspace=0.15
fig, axes = plt.subplots(1, 3, figsize=figsize)
for a, (ax, neuron_property) in  enumerate(zip(axes.flatten(), neuron_properties)):
    r_val = df_mouse_corr[mask&(df_mouse_corr['Value_Type']==neuron_property)&(df_mouse_corr['Activity Type']=='stimulus')]['corr'].mean()
    corr = plot_mouse(df_filtered_mouse_sorted,area, neuron_property, ax,  markersize=7,
                    alpha=0.8, hue='EV gray screen', palette=v4_cmap, legend=True, linewidth=0.07,
                    r_val=r_val, 
                    hue_norm=(global_min,global_max)
                    )
    ax.set_ylim(top=max(df_filtered_mouse_sorted[df_filtered_mouse_sorted.Area==area]['EV'].max(),df_filtered_mouse_removed[df_filtered_mouse_removed.Area==area]['EV'].max()) +0.05)
    ax.set_yticks([0,0.5])
    if a==0 or a==3:
        ax.set_ylabel('EV fraction', fontsize=6, labelpad=1)
    else: 
        ax.set_yticklabels('')
        ax.set(ylabel='')
    if a!=2:
        ax.legend_.remove()
    else:
        legend = ax.legend(loc=(1.1,0),fontsize=6*0.8)
        legend.set_title(title='EV gray screen',prop={'size':6*0.8})
        legend.get_frame().set_linewidth(0.2)
        ax.legend_.remove()
    ax.set_xlabel(None)
axes[2].set_xlim(left=df_filtered_mouse['1-vs-rest r²'].min()-0.07, right=df_filtered_mouse['1-vs-rest r²'].max()+0.07)
plt.subplots_adjust(hspace=hspace, wspace=wspace)

if save_figs is True:
    plt.savefig(fig_dir +'fig6_mouse_resp_grayscreen_corrs_L23_legend.pdf',transparent=True,bbox_inches='tight')


In [ ]:
# Fig 6C top
area='L2/3'
df_filtered_mouse_removed=df_mouse_removed_pcs_both.sample(4000, random_state=17)
df_filtered_mouse_removed_sorted = df_filtered_mouse_removed.sort_values('EV gray screen')
hspace=0.3
wspace=0.15
figsize=(0.75+0.05,0.75)

fig = plt.figure(figsize=figsize)
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 0.05], wspace=0.05)

# Set up the main plot in the first grid space
ax = fig.add_subplot(gs[0])

r_val = df_mouse_removed_pcs_both[df_mouse_removed_pcs_both['Area']==area].reset_index()[neuron_property].corr(df_mouse_removed_pcs_both[df_mouse_removed_pcs_both['Area']==area].reset_index()['EV'])
corr = plot_mouse(df_filtered_mouse_removed_sorted,area, neuron_property, ax, markersize=7,
                    alpha=0.8, hue='EV gray screen', palette=v4_cmap, r_pos=(0.05,0.99), 
                    linewidth=0.07, r_val=r_val,
                    hue_norm=(df_filtered_mouse_sorted['EV gray screen'].min(),df_filtered_mouse_sorted['EV gray screen'].max())
					)
ax.set_ylim(top=max(df_filtered_mouse_removed_sorted[df_filtered_mouse_removed_sorted.Area==area]['EV'].max(),df_filtered_mouse_removed_sorted[df_filtered_mouse_removed_sorted.Area==area]['EV'].max()) +0.05)
ax.set_yticks([0,0.5])
ax.set(xlim=(-0.1,1))
ax.set_ylabel('EV fraction\nremoved n.v.a.', fontsize=6, labelpad=-0.5)
ax.set_xlabel(None)
ax.set_yticklabels('')
ax.set_xticks(ticks=[0,0.5])
plt.subplots_adjust(hspace=hspace, wspace=wspace+0.9)

ax.set_xlim(left=df_filtered_mouse['1-vs-rest r²'].min()-0.07, right=df_filtered_mouse['1-vs-rest r²'].max()+0.07)
ax.legend_.remove()

# Create the colorbar in the second grid space
norm = Normalize(global_min, global_max)
sm = plt.cm.ScalarMappable(cmap=v4_cmap, norm=norm)
sm.set_array([])
cbar_ax = fig.add_subplot(gs[1])
cbar = fig.colorbar(sm, cax=cbar_ax)
cbar.set_label('EV gray\nscreen', fontsize=4, y=1.2, ha='right', rotation=0)
cbar.ax.tick_params(labelsize=4, pad=0.02)
cbar.ax.spines[:].set_linewidth(0.5)


if save_figs is True:
    plt.savefig(fig_dir +'fig6_mouse_resp_grayscreen_corrs_L23_removed_pcs_ceil_reli.pdf',transparent=True,bbox_inches='tight', dpi=4000)

plt.show()


In [ ]:
# Fig 6B bottom
area='L4'

area_bool = df_mouse_corr['Area']==area
mask = (~filter_bool)&area_bool

figsize=(2.5,0.75)
fig, axes = plt.subplots(1, 3, figsize=figsize)
for a, (ax, neuron_property) in  enumerate(zip(axes.flatten(), neuron_properties)):
    r_val = df_mouse_corr[mask&(df_mouse_corr['Value_Type']==neuron_property)&(df_mouse_corr['Activity Type']=='stimulus')]['corr'].mean()    
    corr = plot_mouse(df_filtered_mouse_sorted,area, neuron_property, ax, markersize=7,linewidth=0.07, 
                    alpha=0.8, hue='EV gray screen', palette=v1_cmap, r_val=r_val,
                    hue_norm=(global_min,global_max)
                    )
    ax.set_ylim(top=max(df_filtered_mouse_sorted[df_filtered_mouse_sorted.Area==area]['EV'].max(),df_filtered_mouse_removed[df_filtered_mouse_removed.Area==area]['EV'].max()) +0.05)
    ax.set_yticks([0,0.5])
    if a==0 or a==3:
        ax.set_ylabel('EV fraction', fontsize=6, labelpad=1)
    else: 
        ax.set_yticklabels('')
        ax.set(ylabel='')
    ax.set_xlabel(neuron_property,labelpad=1)
    if a!=2:
        ax.legend_.remove()
    else:
        legend = ax.legend(loc=(1.1,0),fontsize=6*0.8)
        legend.set_title(title='EV gray screen',prop={'size':6*0.8})
        legend.get_frame().set_linewidth(0.2)
        ax.legend_.remove()

axes[2].set_xlim(left=df_filtered_mouse['1-vs-rest r²'].min()-0.07, right=df_filtered_mouse['1-vs-rest r²'].max()+0.07)
plt.subplots_adjust(hspace=hspace, wspace=wspace)
if save_figs is True:
    plt.savefig(fig_dir +'fig6_mouse_resp_grayscreen_corrs_L4.pdf',transparent=True,bbox_inches='tight')

In [ ]:
# Fig 6C bottom

area='L4'
fontsize=6
figsize=(0.75+0.05,0.75)
fig = plt.figure(figsize=figsize)
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 0.05], wspace=0.05)

# Set up the main plot in the first grid space
ax = fig.add_subplot(gs[0])

r_val = df_mouse_removed_pcs_both[df_mouse_removed_pcs_both['Area']==area].reset_index()[neuron_property].corr(df_mouse_removed_pcs_both[df_mouse_removed_pcs_both['Area']==area].reset_index()['EV'])
corr = plot_mouse(df_filtered_mouse_removed_sorted,area, neuron_property, ax, 
                    alpha=0.8, hue='EV gray screen', palette=v1_cmap, markersize=7, 
                	r_val=r_val, linewidth=0.07,legend=False,
                    hue_norm=(df_filtered_mouse_sorted['EV gray screen'].min(),df_filtered_mouse_sorted['EV gray screen'].max())
                    )
ax.set_ylim(top=max(df_filtered_mouse_removed_sorted[df_filtered_mouse_removed_sorted.Area==area]['EV'].max(),df_filtered_mouse_removed_sorted[df_filtered_mouse_removed_sorted.Area==area]['EV'].max()) +0.05)
ax.set_yticks([0,0.5])
ax.set(xlim=(-0.1,1))
ax.set_ylabel('EV fraction\nremoved n.v.a.', fontsize=fontsize, labelpad=-0.5)
ax.set_xlabel('1-vs-rest r²\nremoved n.v.a.', fontsize=fontsize,labelpad=0)
ax.set_yticklabels('')
ax.set_xticks(ticks=[0,0.5])
plt.subplots_adjust(hspace=hspace, wspace=wspace)
ax.set_xlim(left=df_filtered_mouse['1-vs-rest r²'].min()-0.07, right=df_filtered_mouse['1-vs-rest r²'].max()+0.07)

# Create the colorbar in the second grid space
norm = Normalize(global_min, global_max)
sm = plt.cm.ScalarMappable(cmap=v1_cmap, norm=norm)
sm.set_array([])
cbar_ax = fig.add_subplot(gs[1])
cbar = fig.colorbar(sm, cax=cbar_ax)
cbar.set_label('EV gray\nscreen', fontsize=4, y=1.2, ha='right', rotation=0)
cbar.ax.tick_params(labelsize=4, pad=0.02)
cbar.ax.spines[:].set_linewidth(0.5)

if save_figs is True:
    plt.savefig(fig_dir +'fig6_mouse_resp_grayscreen_corrs_L4_removed_pcs_ceil_reli.pdf',transparent=True,bbox_inches='tight', dpi=4000)

plt.show()

### supplemental

In [ ]:
# Fig Supp 7B top
neuron_property = '1-vs-rest r²'
area='L2/3'

figsize=(2.5,0.75)
hspace=0.3
wspace=0.15



fig = plt.figure(figsize=figsize)
gs = gridspec.GridSpec(1, 4, width_ratios=[1, 1, 1, 0.05], wspace=0.1)
# Set up subplots in the first three grid spaces
axes = [fig.add_subplot(gs[i]) for i in range(3)]

mouse_names = df_mouse_both['Mouse Name'].unique()
for a, (ax, mouse_name) in  enumerate(zip(axes, mouse_names)):
    mini_mouse_df = df_mouse_both[df_mouse_both['Mouse Name']==mouse_name].reset_index()
    
    
    mini_mouse_df_sorted = mini_mouse_df.sort_values('EV gray screen')
    corr = plot_mouse(mini_mouse_df_sorted,area, neuron_property, ax,
                    alpha=0.8, hue='EV gray screen', palette=v4_cmap, markersize=5,
                    hue_norm=(global_min,global_max),
                    downsample_fraction=0.9,
					)
    add_ax_properties_corr(a, ax, df_mouse_both, neuron_property)
    ax.set_title(mouse_name, fontsize=6)
    ax.set_xticklabels('')
    ax.set_xlim(left=df_mouse_both['1-vs-rest r²'].min()-0.07, right=df_mouse_both['1-vs-rest r²'].max()+0.07)

plt.subplots_adjust(hspace=hspace, wspace=wspace)

ax.legend_.remove()

# Create the colorbar in the fourth grid space
norm = Normalize(global_min, global_max)
sm = plt.cm.ScalarMappable(cmap=v4_cmap, norm=norm)
sm.set_array([])
cbar_ax = fig.add_subplot(gs[3])
cbar = fig.colorbar(sm, cax=cbar_ax)
cbar.set_label('EV gray\nscreen', fontsize=5, y=1.25, ha='right', rotation=0)
cbar.ax.tick_params(labelsize=5, pad=0.02)
cbar.ax.spines[:].set_linewidth(0.5)


if save_figs is True:
    plt.savefig(fig_dir +'fig6_supp_mouse_resp_grayscreen_corrs_L23_three_mouse.pdf',transparent=True,bbox_inches='tight')



In [ ]:
#Fig Supp 7B bottom

neuron_property = '1-vs-rest r²'
area='L4'

figsize=(2.5,0.75)
fig = plt.figure(figsize=figsize)
gs = gridspec.GridSpec(1, 4, width_ratios=[1, 1, 1, 0.05], wspace=0.1)
# Set up subplots in the first three grid spaces
axes = [fig.add_subplot(gs[i]) for i in range(3)]

mouse_names = df_mouse_both['Mouse Name'].unique()
for a, (ax, mouse_name) in  enumerate(zip(axes, mouse_names)):
    mini_mouse_df = df_mouse_both[df_mouse_both['Mouse Name']==mouse_name].reset_index()
    mini_mouse_df_sorted = mini_mouse_df.sort_values('EV gray screen')
    corr = plot_mouse(mini_mouse_df_sorted,area, neuron_property, ax, 
                    alpha=0.8, hue='EV gray screen', palette=v1_cmap, markersize=5, 
                    hue_norm=(global_min,global_max), downsample_fraction=0.9,)
    add_ax_properties_corr(a, ax, df_mouse_both, neuron_property)
    ax.set_xlabel(neuron_property,labelpad=0)
    ax.set_xlim(left=df_mouse_both['1-vs-rest r²'].min()-0.07, right=df_mouse_both['1-vs-rest r²'].max()+0.07)
plt.subplots_adjust(hspace=hspace, wspace=wspace)

ax.legend_.remove()

# Create the colorbar in the fourth grid space
norm = Normalize(global_min, global_max)
sm = plt.cm.ScalarMappable(cmap=v1_cmap, norm=norm)
sm.set_array([])
cbar_ax = fig.add_subplot(gs[3])
cbar = fig.colorbar(sm, cax=cbar_ax)
cbar.set_label('EV gray\nscreen', fontsize=5, y=1.25, ha='right', rotation=0)
cbar.ax.tick_params(labelsize=5, pad=0.02)
cbar.ax.spines[:].set_linewidth(0.5)
# Adjust the colorbar's vertical position
# Get the original position of the colorbar
pos = cbar.ax.get_position()
cbar.ax.set_position([pos.x0, pos.y0 - 0.1, pos.width, pos.height])

if save_figs is True:
    plt.savefig(fig_dir +'fig6_supp_mouse_resp_grayscreen_corrs_L4_three_mouse.pdf',transparent=True,bbox_inches='tight')




In [ ]:
def fd_bins(x):
    x = np.asarray(x, float)
    if len(x) < 2: return 10
    q25,q75 = np.percentile(x,[25,75]); iqr = q75-q25
    bw = 2*iqr*(len(x)**(-1/3)) if iqr>0 else (x.std(ddof=1)+1e-12)*3.49*(len(x)**(-1/3))
    return max(10, int(np.ceil((x.max()-x.min())/(bw+1e-12))))

In [ ]:
# Fig Supp 7A
colors = {'L2/3': '#72BEB7', 'L4': '#EDAEAE'}
figsize=(1,2)
fontsize=6
fig, axes= plt.subplots(figsize=figsize, nrows=2, sharex=True)
from diptest import diptest
highly_predictable = df_mouse_both[df_mouse_both['EV']>0.4]

for area, ax in zip(df_mouse_both['Area'].unique(),axes.flatten()):
	area_df = highly_predictable[highly_predictable['Area']==area]
	sns.histplot(area_df, x='1-vs-rest r²', ax=ax, color=colors[area], linewidth=0.3)
	sns.despine()
	ax.set_xlabel('1-vs-rest r²', fontsize=fontsize, labelpad=0)
	ax.set_ylabel('Count', fontsize=fontsize, labelpad=0)
	ax.tick_params(axis='both', which='major', labelsize=fontsize, pad=0)
	# perform dip test and plot p-value
	dip, p_val = diptest(area_df['1-vs-rest r²'].to_numpy())
	if p_val<0.001:
		p_val_str = '<0.001'
	elif p_val<0.01:
		p_val_str = '<0.01'
	elif p_val<0.05:
		p_val_str = '<0.05'
	else:
		p_val_str = f'={p_val:.3g}'
	ax.text(0.98, 0.98, f"dip={dip:.3f}\np{p_val_str}",
         transform=ax.transAxes, ha='right', va='top', fontsize=fontsize)
	ax.spines[:].set_linewidth(0.5)
if save_figs:
    plt.savefig(fig_dir +'fig6_supp_mouse_resp_grayscreen_r2_bimodality_diptest.pdf',transparent=True,bbox_inches='tight', dpi=500)

In [ ]:
colors = {'L2/3': '#72BEB7', 'L4': '#EDAEAE'}
figsize=(1,2)
fontsize=6
from diptest import diptest
highly_predictable = df_mouse_both[df_mouse_both['EV']>0.4]
for mouse_name in df_mouse_both['Mouse Name'].unique():
	area_df = highly_predictable[(highly_predictable['Mouse Name']==mouse_name)]
	if len(area_df['EV'])>50:
		dip, p_val = diptest(area_df['1-vs-rest r²'].to_numpy())
		if p_val<0.001:
			p_val_str = '<0.001'
		elif p_val<0.01:
			p_val_str = '<0.01'
		elif p_val<0.05:
			p_val_str = '<0.05'
		else:
			p_val_str = f'={p_val:.3g}'
		print(f"[{mouse_name} - {area}] dip={dip:.3f}, p{p_val_str}")
	

In [ ]:
# Fig Supp 7C top
neuron_property = '1-vs-rest r²'
area='L2/3'

figsize=(2.75,0.75)
hspace=0.3
wspace=0.4
fig = plt.figure(figsize=figsize)


gs = gridspec.GridSpec(1, 4, width_ratios=[1, 1, 1, 0.05], wspace=0.2)

# Set up subplots in the first three grid spaces
axes = [fig.add_subplot(gs[i]) for i in range(3)]

mouse_names = df_mouse_removed_pcs_both['Mouse Name'].unique()
for a, (ax, mouse_name) in  enumerate(zip(axes, mouse_names)):
    mini_mouse_df = df_mouse_removed_pcs_both[df_mouse_removed_pcs_both['Mouse Name']==mouse_name].reset_index()
    mini_mouse_df_sorted = mini_mouse_df.sort_values('EV gray screen')
    corr = plot_mouse(mini_mouse_df_sorted,area, neuron_property, ax, r_pos= (0.05, 1),
                    alpha=0.8, hue='EV gray screen', palette=v4_cmap, markersize=5,
                    hue_norm=(global_min,global_max), downsample_fraction=0.9,
					)
    add_ax_properties_corr(a, ax, df_mouse_removed_pcs_both, neuron_property)
    ax.set_title(mouse_name, fontsize=6, y=1.05)
    ax.set_xticklabels('')
    ax.set_xlim(left=df_mouse_removed_pcs_both['1-vs-rest r²'].min()-0.07, right=df_mouse_removed_pcs_both['1-vs-rest r²'].max()+0.07)

plt.subplots_adjust(hspace=hspace, wspace=wspace)

axes[0].set_ylabel('EV fraction\nremoved n.v.a.', fontsize=6, labelpad=0)
ax.legend_.remove()

# Create the colorbar in the fourth grid space
norm = Normalize(global_min, global_max)
sm = plt.cm.ScalarMappable(cmap=v4_cmap, norm=norm)
sm.set_array([])
cbar_ax = fig.add_subplot(gs[3])
cbar = fig.colorbar(sm, cax=cbar_ax)
cbar.set_label('EV gray\nscreen', fontsize=5, y=1.25, ha='right', rotation=0)
cbar.ax.tick_params(labelsize=5, pad=0.02)
cbar.ax.spines[:].set_linewidth(0.5)

if save_figs is True:
    plt.savefig(fig_dir +'fig6_supp_mouse_resp_grayscreen_corrs_L23_three_mouse_removed_PCs.pdf',transparent=True,bbox_inches='tight')


In [ ]:
# Fig Supp 7C bottom
area='L4'

fig = plt.figure(figsize=(2.75, 0.75))
gs = gridspec.GridSpec(1, 4, width_ratios=[1, 1, 1, 0.05], wspace=0.2)

# Set up subplots in the first three grid spaces
axes = [fig.add_subplot(gs[i]) for i in range(3)]

mouse_names = df_mouse_removed_pcs_both['Mouse Name'].unique()
for a, (ax, mouse_name) in  enumerate(zip(axes, mouse_names)):
    mini_mouse_df = df_mouse_removed_pcs_both[df_mouse_removed_pcs_both['Mouse Name']==mouse_name].reset_index()
    mini_mouse_df_sorted = mini_mouse_df.sort_values('EV gray screen')
    corr = plot_mouse(mini_mouse_df_sorted,area, neuron_property, ax, r_pos= (0.05, 1),
                    alpha=0.8, hue='EV gray screen', palette=v1_cmap, markersize=5,
                    hue_norm=(global_min,global_max), downsample_fraction=0.9)
    add_ax_properties_corr(a, ax, df_mouse_removed_pcs_both, neuron_property)
    ax.set_xlabel(f'{neuron_property}\nremoved n.v.a.',labelpad=0)
    ax.set_xlim(left=df_mouse_removed_pcs_both['1-vs-rest r²'].min()-0.07, right=df_mouse_removed_pcs_both['1-vs-rest r²'].max()+0.07)

plt.subplots_adjust(hspace=hspace, wspace=1)


axes[0].set_ylabel('EV fraction\nremoved n.v.a.', fontsize=6, labelpad=0)
ax.legend_.remove()

# Create the colorbar in the fourth grid space
norm = Normalize(global_min, global_max)
sm = plt.cm.ScalarMappable(cmap=v1_cmap, norm=norm)
sm.set_array([])
cbar_ax = fig.add_subplot(gs[3])
cbar = fig.colorbar(sm, cax=cbar_ax)
cbar.set_label('EV gray\nscreen', fontsize=5, y=1.25, ha='right', rotation=0)
cbar.ax.tick_params(labelsize=5, pad=0.02)
cbar.ax.spines[:].set_linewidth(0.5)

# Adjust the colorbar's vertical position
# Get the original position of the colorbar
pos = cbar.ax.get_position()
cbar.ax.set_position([pos.x0, pos.y0 - 0.1, pos.width, pos.height])

if save_figs is True:
    plt.savefig(fig_dir +'fig6_mouse_supp_resp_grayscreen_corrs_L4_three_mouse_removed_PCs.pdf',transparent=True,bbox_inches='tight')


In [ ]:
# Fig Supp 7D
colors = {'L2/3': '#72BEB7', 'L4': '#EDAEAE'}
figsize=(1,2)
fontsize=6
fig, axes= plt.subplots(figsize=figsize, nrows=2, sharex=True)
from diptest import diptest
highly_predictable = df_mouse_removed_pcs_both[df_mouse_removed_pcs_both['EV']>0.4]

for area, ax in zip(df_mouse_removed_pcs_both['Area'].unique(),axes.flatten()):
	area_df = highly_predictable[highly_predictable['Area']==area]
	sns.histplot(area_df, x='1-vs-rest r²', ax=ax, color=colors[area], linewidth=0.3)
	sns.despine()
	ax.set_xlabel('1-vs-rest r²', fontsize=fontsize, labelpad=0)
	ax.set_ylabel('Count', fontsize=fontsize, labelpad=0)
	ax.tick_params(axis='both', which='major', labelsize=fontsize, pad=0)
	# perform dip test and plot p-value
	dip, p_val = diptest(area_df['1-vs-rest r²'].to_numpy())
	if p_val<0.001:
		p_val_str = '<0.001'
	elif p_val<0.01:
		p_val_str = '<0.01'
	elif p_val<0.05:
		p_val_str = '<0.05'
	else:
		p_val_str = f'={p_val:.3g}'
	ax.text(1.05, 0.99, f"dip={dip:.3f}\np{p_val_str}",
         transform=ax.transAxes, ha='right', va='top', fontsize=fontsize)
	ax.spines[:].set_linewidth(0.5)
if save_figs:
    plt.savefig(fig_dir +'fig6_supp_mouse_resp_grayscreen_r2_bimodality_diptest_removed_pcs.pdf',transparent=True,bbox_inches='tight', dpi=500)

### supplemental

In [ ]:
df_mouse_both_nonan= df_mouse_both.dropna().reset_index()
df_mouse_removed_pcs_both_nonan= df_mouse_removed_pcs_both.dropna().reset_index()
df_mouse_both= df_mouse_both_nonan[df_mouse_both_nonan['EV gray screen'].isin(df_mouse_removed_pcs_both_nonan['EV gray screen'])]
df_mouse_removed_pcs_both = df_mouse_removed_pcs_both_nonan[df_mouse_removed_pcs_both_nonan['EV gray screen'].isin(df_mouse_both_nonan['EV gray screen'])]

In [ ]:
df_mouse_both['removed\nspont. act.']=False
df_mouse_removed_pcs_both['removed\nspont. act.']=True
df_both_comparison = pd.concat([df_mouse_both, df_mouse_removed_pcs_both])

df_both_comparison_visual = pd.concat([df_mouse_both[(df_mouse_both.SNR>2)&(df_mouse_both['split-half r']>0.8)],df_mouse_removed_pcs_both[(df_mouse_both.SNR>2)&(df_mouse_both['split-half r']>0.8)]])
df_both_comparison_nonvisual = pd.concat([df_mouse_both[(df_mouse_both.SNR<2)&(df_mouse_both['split-half r']<0.8)],df_mouse_removed_pcs_both[(df_mouse_both.SNR<2)&(df_mouse_both['split-half r']<0.8)]])
df_both_comparison_mediumvisual = pd.concat([df_mouse_both[(df_mouse_both.SNR<2)|(df_mouse_both['split-half r']<0.8)],df_mouse_removed_pcs_both[(df_mouse_both.SNR<2)|(df_mouse_both['split-half r']<0.8)]])
df_both_comparison_low_SNR = pd.concat([df_mouse_both[(df_mouse_both.SNR<2)],df_mouse_removed_pcs_both[(df_mouse_both.SNR<2)]])
df_both_comparison_high_SNR = pd.concat([df_mouse_both[(df_mouse_both.SNR>2)],df_mouse_removed_pcs_both[(df_mouse_both.SNR>2)]])

trad_threshold = 0.9
df_both_comparison_high_trad_reli = pd.concat([df_mouse_both[(df_mouse_both['split-half r']>trad_threshold)],df_mouse_removed_pcs_both[(df_mouse_both['split-half r']>trad_threshold)]])
df_both_comparison_low_trad_reli = pd.concat([df_mouse_both[(df_mouse_both['split-half r']<trad_threshold)],df_mouse_removed_pcs_both[(df_mouse_both['split-half r']<trad_threshold)]])

In [ ]:
# Fig Supp 7E top 
neuron_property = '1-vs-rest r²'
area='L2/3'
x = 'removed\nspont. act.'
hue='removed\nspont. act.'
sample_size=20
seed=17
fontsize=6
figsize=(2.75,1)
stars_height=0.7
fig, axes = plt.subplots(1, 3, figsize=figsize)
plot_spont_figs(df_both_comparison, area, neuron_property, x, hue, axes,sample_size=sample_size, 
                neuron_property_label='1-vs-rest r²',neuron_labels = ['r>mean','r<mean'],
                    legend=False, show_lineplot=False, height=stars_height)
[axes[a].set_xticklabels([]) for a in range(3)]
if save_figs is True:
    plt.savefig(fig_dir +'fig6_supp_reli_diff_removing_pcs_L23.pdf',transparent=True,bbox_inches='tight')
plt.show()

In [ ]:
# Fig Supp 7E bottom
area='L4'
sample_size=20
seed=17
stars_height=0.7
fig, axes = plt.subplots(1, 3, figsize=figsize)
plot_spont_figs(df_both_comparison, area, neuron_property, x, hue, axes,sample_size=sample_size, 
                neuron_property_label='1-vs-rest r²',neuron_labels = ['r>mean','r<mean'],
                    legend=False, show_lineplot=False, height=stars_height)
[axes[a].set_title(None) for a in range(3)]
if save_figs is True:
    plt.savefig(fig_dir +'fig6_supp_reli_diff_removing_pcs_L4.pdf',transparent=True,bbox_inches='tight')

plt.show()

In [ ]:
# Fig Supp 7F top
neuron_property = 'EV'
area='L2/3'
sample_size=25
seed=17
fontsize=6
fig, axes = plt.subplots(1, 3, figsize=figsize)
threshold=0.4
height=0.9
plot_spont_figs(df_both_comparison, area, neuron_property, x, hue, axes, threshold=threshold,sample_size=sample_size,
                neuron_property_label='EV fraction',neuron_labels = [f'EV>{threshold}',f'EV<{threshold}'],height=height,
                    legend=False, filter_only_high_neurons=True)
[axes[a].set_xticklabels([]) for a in range(3)]
if save_figs is True:
    plt.savefig(fig_dir +'fig6_supp_evar_diff_removing_pcs_L23.pdf',transparent=True,bbox_inches='tight')
plt.show()

In [ ]:
# Fig Supp 7F bottom
neuron_property = 'EV'
area='L4'
sample_size=25
seed=17
fontsize=6
fig, axes = plt.subplots(1, 3, figsize=figsize)
threshold=0.45
height=0.8
plot_spont_figs(df_both_comparison, area, neuron_property, x, hue, axes, threshold=threshold,sample_size=sample_size,
                neuron_property_label='EV fraction',neuron_labels = [f'EV>{threshold}',f'EV<{threshold}'],height=height,
                    legend=False, filter_only_high_neurons=True)
[axes[a].set_title(None) for a in range(3)]
if save_figs is True:
    plt.savefig(fig_dir +'fig6_supp_evar_diff_removing_pcs_L4.pdf',transparent=True,bbox_inches='tight')
plt.show()

## Monkey L neuron properties

### plotting

In [ ]:
monkey_name = 'L'
monkey_stats = all_monkey_stats[monkey_name]
df_monkey_ = make_monkey_df_neuron_properties(monkey_stats, dataset_types=list(monkey_stats.keys()))
df_monkey = df_monkey_[df_monkey_.control_shuffle==False]
df_monkey_corr=make_corr_df_monkey(monkey_stats, df_monkey)




In [ ]:
# Fig 6D top
area = 'V4'
x= 'Value_Type'
y = 'corr'
hue = 'Activity Type'

plt.rc("hatch", linewidth=0.5)
hatch_size=5
xtick_labels = ['max r²', '1-vs-rest r²','SNR', 'split-half r',r'$\text{var}_{\text{time}}$',r'$\text{var}_{\text{repeats}}$',r'$\text{var}_{\text{stim}}$', ]
fig, ax = plt.subplots(figsize=(4,0.65))
mask, order = plot_corr_bars(df_monkey_corr, ax, area=area, hatch_size=hatch_size)
ax.set_xticks(ax.get_xticks(), labels=xtick_labels, rotation=0, ha='center', fontsize=6)
ax.set_ylabel('corr w/ EV fraction', fontsize=6, loc='center')
hue_order = df_monkey_corr[mask][hue].unique()

df_for_stats = df_monkey_corr[mask]
res = pairwise_within_x_using_your_tests(
    df_for_stats, x=x, hue=hue, y=y,
    x_order=order, hue_order=hue_order,
    compare='ref',          # each bin vs the first bin (5 ms)
    mc_method='holm',
    perm_t=True, perm_type='ind', hierarchical=False,
    central_tendency='median', num_permutations=10000,
    mouse_or_date='Date'
)

centers = np.arange(len(order))
n_h = len(hue_order)
# approximate seaborn offsets for grouped violins
width = 0.9
step = width / n_h if n_h > 0 else 0
start = -width/2 + step/2
offsets = [start + k*step for k in range(n_h)]
hue_to_offset = {h: offsets[i] for i, h in enumerate(hue_order)}
stars_fontsize = 5
# plot stars
for x_idx, x_i in enumerate(order):
	pairs_ = res[x_i]['pairs']
	for (h1, h2) in pairs_:
		if (x_i == 'Variance across stimuli') & (((h1, h2) ==('stimulus', 'lights off')) or (h1, h2) ==('gray screen', 'lights off')):
			continue
		pval = res[x_i]['p_adj'][res[x_i]['pairs'].index((h1,h2))] if (h1,h2) in res[x_i]['pairs'] else (res[x_i]['p_adj'][res[x_i]['pairs'].index((h2,h1))] if (h2,h1) in res[x_i]['pairs'] else None)
		stars = p_to_stars(pval)
		subdf = df_for_stats[(df_for_stats[x]==x_i)&(df_for_stats[hue].isin([h1,h2]))]
		x_center = x_idx + 0.5*(hue_to_offset[h1] + hue_to_offset[h2])
		y_pos = subdf[y].max() + 0.01
		if (h1, h2) ==('stimulus', 'lights off'):
			#place higher and plot bar under
			y_pos += 0.18
			ax.plot([hue_to_offset[h1]+x_idx, hue_to_offset[h2]+x_idx], [y_pos, y_pos], color='k', lw=0.5,
           		clip_on=False)
			y_pos += 0.02
		if stars == 'n.s.':
			fontsize_=stars_fontsize*0.8
			# give color of gray
			color_='gray'
		else:
			fontsize_=stars_fontsize
			y_pos -= 0.05
			color_='k'
		ax.text(x_center, y_pos, stars, ha='center', va='bottom', fontsize=fontsize_, color=color_)
legend = ax.legend(loc=(0.82,0.75), fontsize=6*0.7)
legend.get_frame().set_linewidth(0.2)
ax.set_ylim(top=1, bottom=-0.05)

v4_mask = mask.copy()

if save_figs is True:
    plt.savefig(fig_dir +f'fig6_monkey_{monkey_name}_resp_grayscreen_RS_corrs_V4.pdf',transparent=True,bbox_inches='tight')
plt.show()

In [ ]:
# Fig 6D bottom
plt.rc("hatch", linewidth=0.5)
hatch_size=5
xtick_labels = ['max r²', '1-vs-rest r²','SNR', 'split-half r',r'$\text{var}_{\text{time}}$',r'$\text{var}_{\text{repeats}}$',r'$\text{var}_{\text{stim}}$', ]
area = 'V1'
fig, ax = plt.subplots(figsize=(4,0.65))
mask, order = plot_corr_bars(df_monkey_corr, ax, area=area, hatch_size=hatch_size)
ax.set_xticks(ax.get_xticks(), labels=xtick_labels, rotation=0, ha='center', fontsize=6)
ax.set_ylabel('corr w/ EV fraction', fontsize=6, loc='center')


df_for_stats = df_monkey_corr[mask]
res = pairwise_within_x_using_your_tests(
    df_for_stats, x=x, hue=hue, y=y,
    x_order=order, hue_order=hue_order,
    compare='ref',          # each bin vs the first bin (5 ms)
    mc_method='holm',
    perm_t=True, perm_type='ind', hierarchical=False,
    central_tendency='median', num_permutations=10000,
    mouse_or_date='Date'
)

centers = np.arange(len(order))
n_h = len(hue_order)
# approximate seaborn offsets for grouped violins
width = 0.9
step = width / n_h if n_h > 0 else 0
start = -width/2 + step/2
offsets = [start + k*step for k in range(n_h)]
hue_to_offset = {h: offsets[i] for i, h in enumerate(hue_order)}

# plot stars
for x_idx, x_i in enumerate(order):
	pairs_ = res[x_i]['pairs']
	for (h1, h2) in pairs_:
		if (x_i == 'Variance across stimuli') & (((h1, h2) ==('stimulus', 'lights off')) or (h1, h2) ==('gray screen', 'lights off')):
			continue
		pval = res[x_i]['p_adj'][res[x_i]['pairs'].index((h1,h2))] if (h1,h2) in res[x_i]['pairs'] else (res[x_i]['p_adj'][res[x_i]['pairs'].index((h2,h1))] if (h2,h1) in res[x_i]['pairs'] else None)
		stars = p_to_stars(pval)
		subdf = df_for_stats[(df_for_stats[x]==x_i)&(df_for_stats[hue].isin([h1,h2]))]
		x_center = x_idx + 0.5*(hue_to_offset[h1] + hue_to_offset[h2])
		y_pos = subdf[y].max() + 0.01
		if (h1, h2) ==('stimulus', 'lights off'):
			#place higher and plot bar under
			y_pos += 0.18
			ax.plot([hue_to_offset[h1]+x_idx, hue_to_offset[h2]+x_idx], [y_pos, y_pos], color='k', lw=0.5,
           		clip_on=False)
			y_pos += 0.02
		if stars == 'n.s.':
			fontsize_=stars_fontsize*0.8
			# give color of gray
			color_='gray'
		else:
			fontsize_=stars_fontsize
			y_pos -= 0.05
			color_='k'
		ax.text(x_center, y_pos, stars, ha='center', va='bottom', fontsize=fontsize_, color=color_)
legend = ax.legend(loc=(0.82,0.75), fontsize=6*0.7)
legend.get_frame().set_linewidth(0.2)
ax.set_ylim(top=1)
import copy
v1_mask = mask.copy()

if save_figs is True:
    plt.savefig(fig_dir +f'fig6_monkey_{monkey_name}_resp_grayscreen_RS_corrs_V1.pdf',transparent=True,bbox_inches='tight')
plt.show()

In [ ]:
df_monkey_both=pd.merge(df_monkey[df_monkey['Activity Type']=='stimulus'],df_monkey[df_monkey['Activity Type']=='gray screen'].rename(columns={'EV':'EV gray screen','max r² val':'max r² val gray screen'}), on=['Date', 'Area',
    'Direction', 'SNR', 'split-half r', '1-vs-rest r²','control_shuffle'])
df_monkey_both_sorted = df_monkey_both.sort_values('EV gray screen')

# Calculate global min and max for consistent normalization
global_min = df_monkey_both_sorted['EV gray screen'].min()
global_max = df_monkey_both_sorted['EV gray screen'].max()


all_df_monkey_boths = pd.DataFrame()
df_monkey_both_sorted['Monkey Name'] = monkey_name
all_df_monkey_boths  = pd.concat([all_df_monkey_boths, df_monkey_both_sorted])

In [ ]:
# Fig 6E top
neuron_properties = ['max r² val', 'split-half r','1-vs-rest r²']
area='V4'
figsize=(2.5,0.75)
hspace=0.3
wspace=0.15
fig, axes = plt.subplots(1, 3, figsize=figsize)
for a, (ax, neuron_property) in  enumerate(zip(axes.flatten(), neuron_properties)):
    r_val = df_monkey_corr[v4_mask&(df_monkey_corr['Value_Type']==neuron_property)&(df_monkey_corr['Activity Type']=='stimulus')]['corr'].mean()
    corr = plot_date(df_monkey_both_sorted,area, neuron_property, ax, r_loc=(0.05,0.85),s=8,
                    alpha=0.8, hue='EV gray screen', palette=v4_cmap, 
                    hue_norm=(global_min,global_max), linewidth=0.07, r_val=r_val)
    ax.set_ylim(top=1)
    ax.set_yticks([0,0.5,1])
    if a==0 or a==3:
        ax.set_ylabel('EV fraction', fontsize=6, labelpad=1)
    else: 
        ax.set_yticklabels('')
        ax.set(ylabel='')
    if a!=2:
        ax.legend_.remove()
    else:
        legend = ax.legend(loc=(1.1,0),fontsize=6*0.8)
        legend.set_title(title='EV gray screen',prop={'size':6*0.8})
        legend.get_frame().set_linewidth(0.2)
        ax.legend_.remove()
    ax.set_xlabel(None)
axes[0].set_xticks(ticks=[0,0.5])


plt.subplots_adjust(hspace=hspace, wspace=wspace)
# Create a colorbar with the custom colormap
norm = Normalize(global_min,global_max)
sm = plt.cm.ScalarMappable(cmap=v4_cmap, norm=norm)
sm.set_array([])  # Required for matplotlib to recognize the mappable object
cbar = plt.colorbar(sm, ax=ax, pad=0.02, shrink=0.9)
cbar.set_label('EV gray\nscreen', fontsize=4,y=1.25,ha='right', rotation=0)  # Customize label as needed
cbar.ax.tick_params(labelsize=4, pad=0.02)
cbar.ax.spines[:].set_linewidth(0.5)

# Adjust the colorbar's vertical position
# Get the original position of the colorbar
pos = cbar.ax.get_position()
cbar.ax.set_position([pos.x0, pos.y0 - 0.04, pos.width, pos.height])


if save_figs is True:
    plt.savefig(fig_dir +'fig6_monkey_resp_grayscreen_corrs_V4.pdf',transparent=True,bbox_inches='tight', dpi=4000)

In [ ]:
# Fig 6E bottom
area='V1'

fig, axes = plt.subplots(1, 3, figsize=figsize)
for a, (ax, neuron_property) in  enumerate(zip(axes.flatten(), neuron_properties)):
    r_val = df_monkey_corr[v1_mask&(df_monkey_corr['Value_Type']==neuron_property)&(df_monkey_corr['Activity Type']=='stimulus')]['corr'].mean()
    corr = plot_date(df_monkey_both_sorted,area, neuron_property, ax, r_loc=(0.05,0.85),s=8,
                    alpha=0.8, hue='EV gray screen', palette=v1_cmap, 
                    r_val=r_val, linewidth=0.07, hue_norm=(global_min,global_max))
    ax.set_ylim(top=1)
    ax.set_yticks([0,0.5,1])
    if a==0 or a==3:
        ax.set_ylabel('EV fraction', fontsize=6, labelpad=1)
    else: 
        ax.set_yticklabels('')
        ax.set(ylabel='')
    ax.legend_.remove()
    
axes[0].set_xticks(ticks=[0,0.5])
plt.subplots_adjust(hspace=hspace, wspace=wspace)

# Create a colorbar with the custom colormap
norm = Normalize(global_min,global_max)
sm = plt.cm.ScalarMappable(cmap=v1_cmap, norm=norm)
sm.set_array([])  # Required for matplotlib to recognize the mappable object
cbar = plt.colorbar(sm, ax=ax, pad=0.02, shrink=0.9)
cbar.set_label('EV gray\nscreen', fontsize=4,y=1.25,ha='right', rotation=0)  # Customize label as needed
cbar.ax.tick_params(labelsize=4, pad=0.02)
cbar.ax.spines[:].set_linewidth(0.5)

# Adjust the colorbar's vertical position
# Get the original position of the colorbar
pos = cbar.ax.get_position()
cbar.ax.set_position([pos.x0, pos.y0 - 0.04, pos.width, pos.height])

if save_figs is True:
    plt.savefig(fig_dir +'fig6_monkey_resp_grayscreen_corrs_V1.pdf',transparent=True,bbox_inches='tight', dpi=4000)

## Monkey L RF overlap

### plotting

In [ ]:
percent_over=80
percent_under=10
target_x_n=14
df_all = pd.DataFrame([])
condition_types=['SNR', 'SNR_spont','RS','RF_thin','RF_large']
for condition_type in condition_types:
    df_all=pd.concat([df_all,create_df_overlaps(monkey_stats, condition_type, target_x_n, percent_over, percent_under)])
df = df_all[df_all.control_shuffle==False]
df_filtered_all= df_all.groupby(['Activity_Type','Condition_Type','Date', 'Area', 'trad_reli','SNR', 'normal_EV', 'overlap_type', 'Pred_Type', 'target_elec_id','control_shuffle']).agg({'EV':'mean'}).reset_index()
df_filtered_ = df_filtered_all[df_filtered_all.control_shuffle==False]

In [ ]:
# Fig 6F
condition_type = 'SNR'
date='090817'
refelec=187
fig, axes= plt.subplots(2,1, figsize=(1,1.5))
plot_cell_rf_overlaps(df_=df, refelec=refelec, condition_type=condition_type, date=date, axes=axes)
plt.subplots_adjust(hspace=0.08)
if save_figs is True:
    plt.savefig(fig_dir +'fig6_monkey_RF_sample_V4.pdf',transparent=True,bbox_inches='tight')


In [ ]:
#Fig 6H
condition_type = 'SNR'
date='090817'
refelec=810
fig, axes= plt.subplots(2,1, figsize=(1,1.5))
plot_cell_rf_overlaps(df_=df, refelec=refelec, condition_type=condition_type, date=date, axes=axes)
plt.subplots_adjust(hspace=0.08)
if save_figs is True:
    plt.savefig(fig_dir +'fig6_monkey_RF_sample_V1.pdf',transparent=True,bbox_inches='tight')


In [ ]:
# Fig 6G
area='V4'
ax = plot_rf_overlap_performance(df_filtered_all, area, legend_loc=(0.37,0.7))
if save_figs is True:
    plt.savefig(fig_dir +'fig6_monkey_overlap_evars_V4.pdf',transparent=True,bbox_inches='tight')


In [ ]:
# Fig 6I
area='V1'
plot_rf_overlap_performance(df_filtered_all, area, gap=-0.5, legend_loc=(0.37,0.7))
if save_figs is True:
    plt.savefig(fig_dir +'fig6_monkey_overlap_evars_V1.pdf',transparent=True,bbox_inches='tight')


## Monkey A neuron properties

### plotting

In [ ]:
monkey_name = 'A'
monkey_stats = all_monkey_stats[monkey_name]
df_monkey_ = make_monkey_df_neuron_properties(monkey_stats, dataset_types=list(monkey_stats.keys()))
df_monkey = df_monkey_[df_monkey_.control_shuffle==False]
df_monkey_corr=make_corr_df_monkey(monkey_stats, df_monkey)

In [ ]:
# Fig Supp 8 A top
x= 'Value_Type'
y = 'corr'
hue = 'Activity Type'
area = 'V4'
plt.rc("hatch", linewidth=0.5)
hatch_size=5
xtick_labels = ['max r²', '1-vs-rest r²','SNR', 'split-half r',r'$\text{var}_{\text{time}}$',r'$\text{var}_{\text{repeats}}$',r'$\text{var}_{\text{stim}}$', ]
fig, ax = plt.subplots(figsize=(4,0.65))
mask, order = plot_corr_bars(df_monkey_corr, ax, area=area, hatch_size=hatch_size, x=x, hue=hue, y=y)
ax.set_xticks(ax.get_xticks(), labels=xtick_labels, rotation=0, ha='center', fontsize=6)
ax.set_ylabel('corr w/ EV fraction', fontsize=6, loc='center')
hue_order = df_monkey_corr[mask][hue].unique()



df_for_stats = df_monkey_corr[mask]
res = pairwise_within_x_using_your_tests(
    df_for_stats, x=x, hue=hue, y=y,
    x_order=order, hue_order=hue_order,
    compare='ref',          # each bin vs the first bin (5 ms)
    mc_method='holm',
    perm_t=True, perm_type='ind', hierarchical=False,
    central_tendency='median', num_permutations=10000,
    mouse_or_date='Date'
)

centers = np.arange(len(order))
n_h = len(hue_order)
# approximate seaborn offsets for grouped violins
width = 0.9
step = width / n_h if n_h > 0 else 0
start = -width/2 + step/2
offsets = [start + k*step for k in range(n_h)]
hue_to_offset = {h: offsets[i] for i, h in enumerate(hue_order)}
stars_fontsize = 5
# plot stars
for x_idx, x_i in enumerate(order):
	pairs_ = res[x_i]['pairs']
	for (h1, h2) in pairs_:
		if (x_i == 'Variance across stimuli') & (((h1, h2) ==('stimulus', 'lights off')) or (h1, h2) ==('gray screen', 'lights off')):
			continue
		pval = res[x_i]['p_adj'][res[x_i]['pairs'].index((h1,h2))] if (h1,h2) in res[x_i]['pairs'] else (res[x_i]['p_adj'][res[x_i]['pairs'].index((h2,h1))] if (h2,h1) in res[x_i]['pairs'] else None)
		stars = p_to_stars(pval)
		subdf = df_for_stats[(df_for_stats[x]==x_i)&(df_for_stats[hue].isin([h1,h2]))]
		x_center = x_idx + 0.5*(hue_to_offset[h1] + hue_to_offset[h2])
		y_pos = subdf[y].max() + 0.01
		if (h1, h2) ==('stimulus', 'lights off'):
			#place higher and plot bar under
			y_pos += 0.18
			ax.plot([hue_to_offset[h1]+x_idx, hue_to_offset[h2]+x_idx], [y_pos, y_pos], color='k', lw=0.5,
           		clip_on=False)
			y_pos += 0.02
		if stars == 'n.s.':
			fontsize_=stars_fontsize*0.8
			# give color of gray
			color_='gray'
		else:
			fontsize_=stars_fontsize
			y_pos -= 0.05
			color_='k'
		ax.text(x_center, y_pos, stars, ha='center', va='bottom', fontsize=fontsize_, color=color_)
legend = ax.legend(loc=(0.82,0.75), fontsize=6*0.7)
legend.get_frame().set_linewidth(0.2)
ax.set_ylim(top=1)
import copy
v4_mask = mask.copy()

if save_figs is True:
    plt.savefig(fig_dir +f'fig6_monkey_{monkey_name}_resp_grayscreen_RS_corrs_{area}.pdf',transparent=True,bbox_inches='tight')
plt.show()

In [ ]:
# Fig Supp 8 A bottom

x= 'Value_Type'
y = 'corr'
hue = 'Activity Type'
area = 'V1'
plt.rc("hatch", linewidth=0.5)
hatch_size=5
xtick_labels = ['max r²', '1-vs-rest r²','SNR', 'split-half r',r'$\text{var}_{\text{time}}$',r'$\text{var}_{\text{repeats}}$',r'$\text{var}_{\text{stim}}$', ]
fig, ax = plt.subplots(figsize=(4,0.65))
mask, order = plot_corr_bars(df_monkey_corr, ax, area=area, hatch_size=hatch_size, x=x, hue=hue, y=y)
ax.set_xticks(ax.get_xticks(), labels=xtick_labels, rotation=0, ha='center', fontsize=6)
ax.set_ylabel('corr w/ EV fraction', fontsize=6, loc='center')
hue_order = df_monkey_corr[mask][hue].unique()



df_for_stats = df_monkey_corr[mask]
res = pairwise_within_x_using_your_tests(
    df_for_stats, x=x, hue=hue, y=y,
    x_order=order, hue_order=hue_order,
    compare='ref',          # each bin vs the first bin (5 ms)
    mc_method='holm',
    perm_t=True, perm_type='ind', hierarchical=False,
    central_tendency='median', num_permutations=10000,
    mouse_or_date='Date'
)

centers = np.arange(len(order))
n_h = len(hue_order)
# approximate seaborn offsets for grouped violins
width = 0.9
step = width / n_h if n_h > 0 else 0
start = -width/2 + step/2
offsets = [start + k*step for k in range(n_h)]
hue_to_offset = {h: offsets[i] for i, h in enumerate(hue_order)}
stars_fontsize = 5
# plot stars
for x_idx, x_i in enumerate(order):
	pairs_ = res[x_i]['pairs']
	for (h1, h2) in pairs_:
		if (x_i == 'Variance across stimuli') & (((h1, h2) ==('stimulus', 'lights off')) or (h1, h2) ==('gray screen', 'lights off')):
			continue
		pval = res[x_i]['p_adj'][res[x_i]['pairs'].index((h1,h2))] if (h1,h2) in res[x_i]['pairs'] else (res[x_i]['p_adj'][res[x_i]['pairs'].index((h2,h1))] if (h2,h1) in res[x_i]['pairs'] else None)
		stars = p_to_stars(pval)
		subdf = df_for_stats[(df_for_stats[x]==x_i)&(df_for_stats[hue].isin([h1,h2]))]
		x_center = x_idx + 0.5*(hue_to_offset[h1] + hue_to_offset[h2])
		y_pos = subdf[y].max() + 0.01
		if (h1, h2) ==('stimulus', 'lights off'):
			#place higher and plot bar under
			y_pos += 0.18
			ax.plot([hue_to_offset[h1]+x_idx, hue_to_offset[h2]+x_idx], [y_pos, y_pos], color='k', lw=0.5,
           		clip_on=False)
			y_pos += 0.02
		if stars == 'n.s.':
			fontsize_=stars_fontsize*0.8
			# give color of gray
			color_='gray'
		else:
			fontsize_=stars_fontsize
			y_pos -= 0.05
			color_='k'
		ax.text(x_center, y_pos, stars, ha='center', va='bottom', fontsize=fontsize_, color=color_)
legend = ax.legend(loc=(0.82,0.75), fontsize=6*0.7)
legend.get_frame().set_linewidth(0.2)
ax.set_ylim(top=1)
import copy
v1_mask = mask.copy()

if save_figs is True:
    plt.savefig(fig_dir +f'fig6_monkey_{monkey_name}_resp_grayscreen_corrs_RS_{area}.pdf',transparent=True,bbox_inches='tight')
plt.show()

In [ ]:
df_monkey_both=pd.merge(df_monkey[df_monkey['Activity Type']=='stimulus'],df_monkey[df_monkey['Activity Type']=='gray screen'].rename(columns={'EV':'EV gray screen','max r² val':'max r² val gray screen'}), on=['Date', 'Area',
    'Direction', 'SNR', 'split-half r', '1-vs-rest r²','control_shuffle'])
df_monkey_both_sorted = df_monkey_both.sort_values('EV gray screen')

# Calculate global min and max for consistent normalization
global_min = df_monkey_both_sorted['EV gray screen'].min()
global_max = df_monkey_both_sorted['EV gray screen'].max()

df_monkey_both_sorted['Monkey Name'] = monkey_name
all_df_monkey_boths  = pd.concat([all_df_monkey_boths, df_monkey_both_sorted])

In [ ]:
# Fig Supp 8 B top

neuron_properties = ['max r² val', 'split-half r','1-vs-rest r²']
area='V4'
figsize=(2.5,0.75)
hspace=0.3
wspace=0.15
fig, axes = plt.subplots(1, 3, figsize=figsize)
for a, (ax, neuron_property) in  enumerate(zip(axes.flatten(), neuron_properties)):
    r_val = df_monkey_corr[v4_mask&(df_monkey_corr['Value_Type']==neuron_property)&(df_monkey_corr['Activity Type']=='stimulus')]['corr'].mean()
    corr = plot_date(df_monkey_both_sorted,area, neuron_property, ax, r_loc=(0.05,0.85),s=8,
                    alpha=0.8, hue='EV gray screen', palette=v4_cmap, 
                    hue_norm=(global_min,global_max), linewidth=0.07, r_val=r_val)
    ax.set_ylim(top=1)
    ax.set_yticks([0,0.5,1])
    if a==0 or a==3:
        ax.set_ylabel('EV fraction', fontsize=6, labelpad=1)
    else: 
        ax.set_yticklabels('')
        ax.set(ylabel='')
    if a!=2:
        ax.legend_.remove()
    else:
        legend = ax.legend(loc=(1.1,0),fontsize=6*0.8)
        legend.set_title(title='EV gray screen',prop={'size':6*0.8})
        legend.get_frame().set_linewidth(0.2)
        ax.legend_.remove()
    ax.set_xlabel(None)
axes[0].set_xticks(ticks=[0,0.5])


plt.subplots_adjust(hspace=hspace, wspace=wspace)
# Create a colorbar with the custom colormap
norm = Normalize(global_min,global_max)
sm = plt.cm.ScalarMappable(cmap=v4_cmap, norm=norm)
sm.set_array([])  # Required for matplotlib to recognize the mappable object
cbar = plt.colorbar(sm, ax=ax, pad=0.02, shrink=0.9)
cbar.set_label('EV gray\nscreen', fontsize=4,y=1.25,ha='right', rotation=0)  # Customize label as needed
cbar.ax.tick_params(labelsize=4, pad=0.02)
cbar.ax.spines[:].set_linewidth(0.5)

# Adjust the colorbar's vertical position
# Get the original position of the colorbar
pos = cbar.ax.get_position()
cbar.ax.set_position([pos.x0, pos.y0 - 0.04, pos.width, pos.height])

if save_figs is True:
    plt.savefig(fig_dir + f'fig6_monkey_{monkey_name}_resp_grayscreen_corrs_V4.pdf',transparent=True,bbox_inches='tight', dpi=4000)

In [ ]:
# Fig Supp 8 B bottom
area='V1'

fig, axes = plt.subplots(1, 3, figsize=figsize)
for a, (ax, neuron_property) in  enumerate(zip(axes.flatten(), neuron_properties)):
    r_val = df_monkey_corr[v1_mask&(df_monkey_corr['Value_Type']==neuron_property)&(df_monkey_corr['Activity Type']=='stimulus')]['corr'].mean()
    corr = plot_date(df_monkey_both_sorted,area, neuron_property, ax, r_loc=(0.05,0.85),s=8,
                    alpha=0.8, hue='EV gray screen', palette=v1_cmap, 
                    r_val=r_val, linewidth=0.07, hue_norm=(global_min,global_max))
    ax.set_ylim(top=1)
    ax.set_yticks([0,0.5,1])
    if a==0 or a==3:
        ax.set_ylabel('EV fraction', fontsize=6, labelpad=1)
    else: 
        ax.set_yticklabels('')
        ax.set(ylabel='')
    ax.legend_.remove()
    
axes[0].set_xticks(ticks=[0,0.5])
plt.subplots_adjust(hspace=hspace, wspace=wspace)

# Create a colorbar with the custom colormap
norm = Normalize(global_min,global_max)
sm = plt.cm.ScalarMappable(cmap=v1_cmap, norm=norm)
sm.set_array([])  # Required for matplotlib to recognize the mappable object
cbar = plt.colorbar(sm, ax=ax, pad=0.02, shrink=0.9)
cbar.set_label('EV gray\nscreen', fontsize=4,y=1.25,ha='right', rotation=0)  # Customize label as needed
cbar.ax.tick_params(labelsize=4, pad=0.02)
cbar.ax.spines[:].set_linewidth(0.5)

# Adjust the colorbar's vertical position
# Get the original position of the colorbar
pos = cbar.ax.get_position()
cbar.ax.set_position([pos.x0, pos.y0 - 0.04, pos.width, pos.height])

if save_figs is True:
    plt.savefig(fig_dir + f'fig6_monkey_{monkey_name}_resp_grayscreen_corrs_V1.pdf',transparent=True,bbox_inches='tight', dpi=4000)

## Monkey A RF overlap

### plotting

In [ ]:
monkey_name = 'A'
monkey_stats = all_monkey_stats[monkey_name]
df_monkey_ = make_monkey_df_neuron_properties(monkey_stats, dataset_types=list(monkey_stats.keys()), get_variances=False)
df_monkey = df_monkey_[df_monkey_.control_shuffle==False]
df_monkey_corr=make_corr_df_monkey(monkey_stats, df_monkey)

percent_over=80
percent_under=10
target_x_n=14
df_all = pd.DataFrame([])
condition_types=['SNR', 'SNR_spont','RF_thin','RF_large', 'RF_thin_spont', 'RF_large_spont']
for condition_type in condition_types:
    print(condition_type)
    df_all=pd.concat([df_all,create_df_overlaps(monkey_stats, condition_type, target_x_n, percent_over, percent_under, monkey='A')])
df = df_all[df_all.control_shuffle==False]
df_filtered_all= df_all.groupby(['Activity_Type','Condition_Type','Date', 'Area', 'trad_reli','SNR', 'normal_EV', 'overlap_type', 'Pred_Type', 'target_elec_id','control_shuffle']).agg({'EV':'mean'}).reset_index()
df_filtered_ = df_filtered_all[df_filtered_all.control_shuffle==False]

In [ ]:
# Fig Supp 8C left
condition_type = 'SNR'
date='041018'
refelec=125
fig, axes= plt.subplots(2,1, figsize=(1,1.5))
plot_cell_rf_overlaps(df_=df, refelec=refelec, condition_type=condition_type, date=date, axes=axes, monkey=monkey_name)
plt.subplots_adjust(hspace=0.08)
if save_figs is True:
    plt.savefig(fig_dir + f'fig6_monkey_{monkey_name}_RF_sample_V4.pdf',transparent=True,bbox_inches='tight')


In [ ]:
#Fig Supp 8D left
condition_type = 'SNR'
date='041018'
refelec=651
fig, axes= plt.subplots(2,1, figsize=(1,1.5))
plot_cell_rf_overlaps(df_=df, refelec=refelec, condition_type=condition_type, date=date, axes=axes, monkey=monkey_name)
plt.subplots_adjust(hspace=0.08)
if save_figs is True:
    plt.savefig(fig_dir + f'fig6_monkey_{monkey_name}_RF_sample_V1.pdf',transparent=True,bbox_inches='tight')


In [ ]:
#Fig Supp 8C right
# sort values by ['image', 'gray screen'] in 'Activity_Type'
df_filtered_ordered = df_filtered_all.sort_values(by=['Activity_Type'], key=lambda x: x.map({'image':0, 'gray screen':1}))
df_filtered_ordered2 = df_filtered_ordered.sort_values(by=['overlap_type'], key=lambda x: x.map({f'<{percent_under}%':0, f'>{percent_over}%':1, 'all':2}))
area='V4'
ax = plot_rf_overlap_performance(df_filtered_ordered2, area, legend_loc=(0.49,0.51))
ax.set_ylim(top=0.75)
if save_figs is True:
    plt.savefig(fig_dir + f'fig6_monkey_{monkey_name}_overlap_evars_V4.pdf',transparent=True,bbox_inches='tight')


In [ ]:
# Fig Supp 8D right
area='V1'
ax = plot_rf_overlap_performance(df_filtered_ordered2, area, gap=-0.5, legend_loc=(0.45,0.51))
ax.set_ylim(top=0.885)
if save_figs is True:
    plt.savefig(fig_dir + f'fig6_monkey_{monkey_name}_overlap_evars_V1.pdf',transparent=True,bbox_inches='tight')


## Monkey D neuron properties

### plotting

In [ ]:
monkey_name = 'D'
D_dataset_types = ['SNR', 'SNR_spont','RS_open']
monkey_stats = all_monkey_stats[monkey_name]
df_monkey_ = make_monkey_df_neuron_properties(monkey_stats, dataset_types=D_dataset_types)
df_monkey = df_monkey_[df_monkey_.control_shuffle==False]
df_monkey_corr=make_corr_df_monkey(monkey_stats, df_monkey)

In [ ]:
# Fig Supp 9A top
area = 'V4'
x= 'Value_Type'
y = 'corr'
hue = 'Activity Type'

plt.rc("hatch", linewidth=0.5)
hatch_size=5
xtick_labels = ['max r²', '1-vs-rest r²','SNR', 'split-half r',r'$\text{var}_{\text{time}}$',r'$\text{var}_{\text{repeats}}$',r'$\text{var}_{\text{stim}}$', ]
fig, ax = plt.subplots(figsize=(3.8,0.65))
mask, order = plot_corr_bars(df_monkey_corr, ax, area=area, hatch_size=hatch_size)
ax.set_xticks(ax.get_xticks(), labels=xtick_labels, rotation=0, ha='center', fontsize=6)
ax.set_ylabel('corr w/ EV fraction', fontsize=6, loc='center')
hue_order = df_monkey_corr[mask][hue].unique()

df_for_stats = df_monkey_corr[mask]
res = pairwise_within_x_using_your_tests(
    df_for_stats, x=x, hue=hue, y=y,
    x_order=order, hue_order=hue_order,
    compare='ref',          # each bin vs the first bin (5 ms)
    mc_method='holm',
    perm_t=True, perm_type='ind', hierarchical=False,
    central_tendency='median', num_permutations=10000,
    mouse_or_date='Date'
)

centers = np.arange(len(order))
n_h = len(hue_order)
# approximate seaborn offsets for grouped violins
width = 0.9
step = width / n_h if n_h > 0 else 0
start = -width/2 + step/2
offsets = [start + k*step for k in range(n_h)]
hue_to_offset = {h: offsets[i] for i, h in enumerate(hue_order)}
stars_fontsize = 5
# plot stars
for x_idx, x_i in enumerate(order):
	pairs_ = res[x_i]['pairs']
	for (h1, h2) in pairs_:
		if (x_i == 'Variance across stimuli') & (((h1, h2) ==('stimulus', 'lights off')) or (h1, h2) ==('gray screen', 'lights off')):
			continue
		pval = res[x_i]['p_adj'][res[x_i]['pairs'].index((h1,h2))] if (h1,h2) in res[x_i]['pairs'] else (res[x_i]['p_adj'][res[x_i]['pairs'].index((h2,h1))] if (h2,h1) in res[x_i]['pairs'] else None)
		stars = p_to_stars(pval)
		subdf = df_for_stats[(df_for_stats[x]==x_i)&(df_for_stats[hue].isin([h1,h2]))]
		x_center = x_idx + 0.5*(hue_to_offset[h1] + hue_to_offset[h2])
		y_pos = subdf[y].max() + 0.01
		# if x_i in ['Variance w/in trial\nacross timepoints','Variance w/in timepoint\nacross trials']:
		# 	y_pos += 0.4
		if (h1, h2) ==('stimulus', 'lights off'):
			#place higher and plot bar under
			y_pos += 0.18
			ax.plot([hue_to_offset[h1]+x_idx, hue_to_offset[h2]+x_idx], [y_pos, y_pos], color='k', lw=0.5,
           		clip_on=False)
			y_pos += 0.02
		if stars == 'n.s.':
			fontsize_=stars_fontsize*0.8
			# give color of gray
			color_='gray'
		else:
			fontsize_=stars_fontsize
			y_pos -= 0.05
			color_='k'
		ax.text(x_center, y_pos, stars, ha='center', va='bottom', fontsize=fontsize_, color=color_)
legend = ax.legend(loc=(0.8,0.95), fontsize=6*0.7)
legend.get_frame().set_linewidth(0.2)
ax.set_ylim(top=1, bottom=None)
v4_mask = mask.copy()

if save_figs is True:
    plt.savefig(fig_dir +f'fig6_monkey_{monkey_name}_resp_grayscreen_RS_corrs_V4.pdf',transparent=True,bbox_inches='tight')
plt.show()

In [ ]:
# Fig Supp 9A bottom
area = 'V1'
x= 'Value_Type'
y = 'corr'
hue = 'Activity Type'

plt.rc("hatch", linewidth=0.5)
hatch_size=5
xtick_labels = ['max r²', '1-vs-rest r²','SNR', 'split-half r',r'$\text{var}_{\text{time}}$',r'$\text{var}_{\text{repeats}}$',r'$\text{var}_{\text{stim}}$', ]
fig, ax = plt.subplots(figsize=(3.8,0.65))
mask, order = plot_corr_bars(df_monkey_corr, ax, area=area, hatch_size=hatch_size)
ax.set_xticks(ax.get_xticks(), labels=xtick_labels, rotation=0, ha='center', fontsize=6)
ax.set_ylabel('corr w/ EV fraction', fontsize=6, loc='center')
hue_order = df_monkey_corr[mask][hue].unique()

df_for_stats = df_monkey_corr[mask]
res = pairwise_within_x_using_your_tests(
    df_for_stats, x=x, hue=hue, y=y,
    x_order=order, hue_order=hue_order,
    compare='ref',          # each bin vs the first bin (5 ms)
    mc_method='holm',
    perm_t=True, perm_type='ind', hierarchical=False,
    central_tendency='median', num_permutations=10000,
    mouse_or_date='Date'
)

centers = np.arange(len(order))
n_h = len(hue_order)
# approximate seaborn offsets for grouped violins
width = 0.9
step = width / n_h if n_h > 0 else 0
start = -width/2 + step/2
offsets = [start + k*step for k in range(n_h)]
hue_to_offset = {h: offsets[i] for i, h in enumerate(hue_order)}
stars_fontsize = 5
# plot stars
for x_idx, x_i in enumerate(order):
	pairs_ = res[x_i]['pairs']
	for (h1, h2) in pairs_:
		if (x_i == 'Variance across stimuli') & (((h1, h2) ==('stimulus', 'lights off')) or (h1, h2) ==('gray screen', 'lights off')):
			continue
		pval = res[x_i]['p_adj'][res[x_i]['pairs'].index((h1,h2))] if (h1,h2) in res[x_i]['pairs'] else (res[x_i]['p_adj'][res[x_i]['pairs'].index((h2,h1))] if (h2,h1) in res[x_i]['pairs'] else None)
		stars = p_to_stars(pval)
		subdf = df_for_stats[(df_for_stats[x]==x_i)&(df_for_stats[hue].isin([h1,h2]))]
		x_center = x_idx + 0.5*(hue_to_offset[h1] + hue_to_offset[h2])
		y_pos = subdf[y].max() + 0.01
		# if x_i in ['Variance w/in trial\nacross timepoints','Variance w/in timepoint\nacross trials']:
		# 	y_pos += 0.45
		if (h1, h2) ==('stimulus', 'lights off'):
			#place higher and plot bar under
			y_pos += 0.18
			# if x_i in ['Variance w/in trial\nacross timepoints','Variance w/in timepoint\nacross trials']:
			# 	y_pos += 0.2
			ax.plot([hue_to_offset[h1]+x_idx, hue_to_offset[h2]+x_idx], [y_pos, y_pos], color='k', lw=0.5,
           		clip_on=False)
			y_pos += 0.02
		if stars == 'n.s.':
			fontsize_=stars_fontsize*0.8
			# give color of gray
			color_='gray'
		else:
			fontsize_=stars_fontsize
			y_pos -= 0.05
			color_='k'
		ax.text(x_center, y_pos, stars, ha='center', va='bottom', fontsize=fontsize_, color=color_)
legend = ax.legend(loc=(0.8,0.99), fontsize=6*0.7)
legend.get_frame().set_linewidth(0.2)
ax.set_ylim(top=1, bottom=None)
v1_mask = mask.copy()

if save_figs is True:
    plt.savefig(fig_dir +f'fig6_monkey_{monkey_name}_resp_grayscreen_RS_corrs_{area}.pdf',transparent=True,bbox_inches='tight')
plt.show()

In [ ]:
df_monkey_both=pd.merge(df_monkey[df_monkey['Activity Type']=='stimulus'],df_monkey[df_monkey['Activity Type']=='gray screen'].rename(columns={'EV':'EV gray screen','max r² val':'max r² val gray screen'}), on=['Date', 'Area',
    'Direction', 'SNR', 'split-half r', '1-vs-rest r²','control_shuffle'])
df_monkey_both_sorted = df_monkey_both.sort_values('EV gray screen')

# Calculate global min and max for consistent normalization
global_min = df_monkey_both_sorted['EV gray screen'].min()
global_max = df_monkey_both_sorted['EV gray screen'].max()


df_monkey_both_sorted['Monkey Name'] = monkey_name
all_df_monkey_boths  = pd.concat([all_df_monkey_boths, df_monkey_both_sorted])

In [ ]:
# Fig Supp 9B top
neuron_properties = ['max r² val', 'split-half r','1-vs-rest r²']
area='V4'
figsize=(2.5,0.75)
fontsize=6
hspace=0.3
wspace=0.15
fig, axes = plt.subplots(1, 3, figsize=figsize)
for a, (ax, neuron_property) in  enumerate(zip(axes.flatten(), neuron_properties)):
    r_val = df_monkey_corr[v4_mask&(df_monkey_corr['Value_Type']==neuron_property)&(df_monkey_corr['Activity Type']=='stimulus')]['corr'].mean()
    corr = plot_date(df_monkey_both_sorted,area, neuron_property, ax, r_loc=(0.05,0.85),s=12,
                    alpha=0.8, hue='EV gray screen', palette=v4_cmap, 
                    hue_norm=(global_min,global_max), linewidth=0.07, r_val=r_val)
    ax.set_ylim(top=0.25, bottom=-0.01)
    # ax.set_yticks([0,0.5,1])
    if a==0 or a==3:
        ax.set_ylabel('EV fraction', fontsize=fontsize, labelpad=1)
    else: 
        ax.set_yticklabels('')
        ax.set(ylabel='')
    if a!=2:
        ax.legend_.remove()
    else:
        legend = ax.legend(loc=(1.1,0),fontsize=fontsize*0.8)
        legend.set_title(title='EV gray screen',prop={'size':fontsize*0.8})
        legend.get_frame().set_linewidth(0.2)
        ax.legend_.remove()
    # ax.set_xlabel(None)
axes[1].set_xticks(ticks=[0,0.5])


plt.subplots_adjust(hspace=hspace, wspace=wspace)
# Create a colorbar with the custom colormap
norm = Normalize(global_min,global_max)
sm = plt.cm.ScalarMappable(cmap=v4_cmap, norm=norm)
sm.set_array([])  # Required for matplotlib to recognize the mappable object
cbar = plt.colorbar(sm, ax=ax, pad=0.02, shrink=0.9)
cbar.set_label('EV gray\nscreen', fontsize=4,y=1.25,ha='right', rotation=0)  # Customize label as needed
cbar.ax.tick_params(labelsize=fontsize*0.8, pad=0.02)
cbar.ax.spines[:].set_linewidth(0.5)

# Adjust the colorbar's vertical position
# Get the original position of the colorbar
pos = cbar.ax.get_position()
cbar.ax.set_position([pos.x0, pos.y0 - 0.04, pos.width, pos.height])


if save_figs is True:
    plt.savefig(fig_dir +f'fig6_monkey_{monkey_name}_resp_grayscreen_corrs_V4.pdf',transparent=True,bbox_inches='tight', dpi=4000)

In [ ]:
# Fig Supp 9B bottom
neuron_properties = ['max r² val', 'split-half r','1-vs-rest r²']
area='V1'
figsize=(2.5,0.75)
# figsize=(5,1.5)
fontsize=6
hspace=0.3
wspace=0.15
fig, axes = plt.subplots(1, 3, figsize=figsize)
for a, (ax, neuron_property) in  enumerate(zip(axes.flatten(), neuron_properties)):
    r_val = df_monkey_corr[v1_mask&(df_monkey_corr['Value_Type']==neuron_property)&(df_monkey_corr['Activity Type']=='stimulus')]['corr'].mean()
    corr = plot_date(df_monkey_both_sorted,area, neuron_property, ax, r_loc=(0.05,0.85),s=12,
                    alpha=0.8, hue='EV gray screen', palette=v1_cmap, 
                    hue_norm=(global_min,global_max), linewidth=0.07, r_val=r_val)
    ax.set_ylim(top=0.35, bottom=-0.01)
    # ax.set_yticks([0,0.5,1])
    if a==0 or a==3:
        ax.set_ylabel('EV fraction', fontsize=fontsize, labelpad=1)
    else: 
        ax.set_yticklabels('')
        ax.set(ylabel='')
    if a!=2:
        ax.legend_.remove()
    else:
        legend = ax.legend(loc=(1.1,0),fontsize=fontsize*0.8)
        legend.set_title(title='EV gray screen',prop={'size':fontsize*0.8})
        legend.get_frame().set_linewidth(0.2)
        ax.legend_.remove()
    # ax.set_xlabel(None)
axes[1].set_xticks(ticks=[0,0.5])


plt.subplots_adjust(hspace=hspace, wspace=wspace)
# Create a colorbar with the custom colormap
norm = Normalize(global_min,global_max)
sm = plt.cm.ScalarMappable(cmap=v1_cmap, norm=norm)
sm.set_array([])  # Required for matplotlib to recognize the mappable object
cbar = plt.colorbar(sm, ax=ax, pad=0.02, shrink=0.9)
cbar.set_label('EV gray\nscreen', fontsize=4,y=1.25,ha='right', rotation=0)  # Customize label as needed
cbar.ax.tick_params(labelsize=fontsize*0.8, pad=0.02)
cbar.ax.spines[:].set_linewidth(0.5)

# Adjust the colorbar's vertical position
# Get the original position of the colorbar
pos = cbar.ax.get_position()
cbar.ax.set_position([pos.x0, pos.y0 - 0.04, pos.width, pos.height])


if save_figs is True:
    plt.savefig(fig_dir + f'fig6_monkey_{monkey_name}_resp_grayscreen_corrs_{area}.pdf',transparent=True,bbox_inches='tight', dpi=4000)

In [ ]:
# print diptest results
highly_predictable = all_df_monkey_boths[all_df_monkey_boths['EV']>0.4]

dip, p_val = diptest(highly_predictable['1-vs-rest r²'].to_numpy())
print(f"Monkey {monkey_name} highly predictable neurons dip test: dip={dip:.3g}, p={p_val:.3g}")

## Monkey $L_{A}$ neuron properties

In [ ]:

# info for subsampled analysis is in results/fig_5

with open(os.path.join(project_root, 'results/fig_5', 'subsample_seeds.json'), 'r') as f:
    subsample_seeds = json.load(f)
    
main_monkey_name = 'L'
main_monkey_stats = all_monkey_stats[main_monkey_name]
subsample_monkey_name = 'A'


stimulus_df_monkey_L_A = pd.DataFrame()
stimulus_df_monkey_L_A_null = pd.DataFrame()
pval_dict_all_seeds = {}
fontsize=7
df_monkey_all_seeds = pd.DataFrame()
df_monkey_corr_L_A_all_seeds = pd.DataFrame()



x= 'Value_Type'
y = 'corr'
hue = 'Activity Type'
area = 'V4'
plt.rc("hatch", linewidth=0.5)
hatch_size=5
xtick_labels = ['max r²', '1-vs-rest r²','SNR', 'split-half r',r'$\text{var}_{\text{time}}$',r'$\text{var}_{\text{repeats}}$',r'$\text{var}_{\text{stim}}$', ]


for seed in subsample_seeds:
	subsampled_monkey_stats_path = os.path.join(project_root,f'results/fig_5/monkey_L_subsampled_to_{subsample_monkey_name}',f'monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_seed{seed}_stats.pkl')
	with open(subsampled_monkey_stats_path, 'rb') as f:
		subsampled_monkey_stats = pickle.load(f)
	all_datataset_types_ = [k for k in subsampled_monkey_stats.keys() if k not in ['meta','monkey_alphas','monkey_alphas_glm','monkey_directionality_alphas']]
	df_monkey_ = make_monkey_df_neuron_properties(subsampled_monkey_stats, dataset_types=all_datataset_types_)
	df_monkey = df_monkey_[df_monkey_.control_shuffle==False]
	df_monkey['subsample_seed'] = seed
	df_monkey_all_seeds = pd.concat([df_monkey_all_seeds, df_monkey])
	df_monkey_corr=make_corr_df_monkey(subsampled_monkey_stats, df_monkey)
	df_monkey_corr['subsample_seed'] = seed
	df_monkey_corr_L_A_all_seeds = pd.concat([df_monkey_corr_L_A_all_seeds, df_monkey_corr])

	order= [e for e in df_monkey_corr.Value_Type.unique() if e!='EV']
	filter_bool = df_monkey_corr['relationship'].isin(['EV_EV'])
	area_bool = df_monkey_corr['Area']==area
	mask = (~filter_bool)&area_bool

	hue_order = df_monkey_corr[mask][hue].unique()
	df_for_stats = df_monkey_corr[mask]
	res = pairwise_within_x_using_your_tests(
		df_for_stats, x=x, hue=hue, y=y,
		x_order=order, hue_order=hue_order,
		compare='ref',          # each bin vs the first bin (5 ms)
		mc_method='holm',
		perm_t=True, perm_type='ind', hierarchical=False,
		central_tendency='median', num_permutations=10000,
		mouse_or_date='Date'
	)
	centers = np.arange(len(order))
	n_h = len(hue_order)

	for x_idx, x_i in enumerate(order):
		if x_i not in pval_dict_all_seeds:
			pval_dict_all_seeds[x_i] = {}
		pairs_ = res[x_i]['pairs']
		for (h1, h2) in pairs_:
			if (x_i == 'Variance across stimuli') & (((h1, h2) ==('stimulus', 'lights off')) or (h1, h2) ==('gray screen', 'lights off')):
				continue
			if (h1, h2) not in pval_dict_all_seeds[x_i]:
				pval_dict_all_seeds	[x_i][(h1, h2)] = []
			pval = res[x_i]['p_adj'][res[x_i]['pairs'].index((h1,h2))] if (h1,h2) in res[x_i]['pairs'] else (res[x_i]['p_adj'][res[x_i]['pairs'].index((h2,h1))] if (h2,h1) in res[x_i]['pairs'] else None)
			pval_dict_all_seeds[x_i][(h1, h2)].append(pval)

# take median over seeds for p_vals_L_A
p_vals_L_A_median = {}
for x_i in pval_dict_all_seeds:
	p_vals_L_A_median[x_i] = {}
	for (h1, h2) in pval_dict_all_seeds[x_i]:
		p_vals_L_A_median[x_i][(h1, h2)] = np.median(pval_dict_all_seeds[x_i][(h1, h2)])
df_corr_mean_L_A = df_monkey_corr_L_A_all_seeds.groupby(['Area', 'Value_Type', 'Ref_property','relationship', 'Activity Type','Date'])['corr'].mean().reset_index()



In [ ]:
# Fig Supp 8E top
x= 'Value_Type'
y = 'corr'
hue = 'Activity Type'
area = 'V4'
plt.rc("hatch", linewidth=0.5)
hatch_size=5
hue_order = ['stimulus','gray screen']
order = ['max r² val', '1-vs-rest r²', 'SNR', 'split-half r',
       'Variance w/in trial\nacross timepoints',
       'Variance w/in timepoint\nacross trials',
       'Variance across stimuli']
xtick_labels = ['max r²', '1-vs-rest r²','SNR', 'split-half r',r'$\text{var}_{\text{time}}$',r'$\text{var}_{\text{repeats}}$',r'$\text{var}_{\text{stim}}$', ]
fig, ax = plt.subplots(figsize=(4,0.65))
mask, _ = plot_corr_bars(df_corr_mean_L_A, ax, area=area, hatch_size=hatch_size, x=x, hue=hue, y=y, order=order, hue_order=hue_order)
ax.set_xticks(ax.get_xticks(), labels=xtick_labels, rotation=0, ha='center', fontsize=6, )
ax.set_ylabel('corr w/ EV fraction', fontsize=6, loc='center')


res = p_vals_L_A_median
centers = np.arange(len(order))
n_h = len(hue_order)
# approximate seaborn offsets for grouped violins
width = 0.9
step = width / n_h if n_h > 0 else 0
start = -width/2 + step/2
offsets = [start + k*step for k in range(n_h)]
hue_to_offset = {h: offsets[i] for i, h in enumerate(hue_order)}
stars_fontsize = 5
# plot stars
for x_idx, x_i in enumerate(order):
	pairs_ = res[x_i]
	for (h1, h2) in pairs_:
		if (x_i == 'Variance across stimuli') & (((h1, h2) ==('stimulus', 'lights off')) or (h1, h2) ==('gray screen', 'lights off')):
			continue
		pval = res[x_i][(h1,h2)] if (h1,h2) in res[x_i] else (res[x_i]['p_adj'][res[x_i].index((h2,h1))] if (h2,h1) in res[x_i]['pairs'] else None)
		stars = p_to_stars(pval)
		subdf = df_for_stats[(df_for_stats[x]==x_i)&(df_for_stats[hue].isin([h1,h2]))]
		x_center = x_idx + 0.5*(hue_to_offset[h1] + hue_to_offset[h2])
		y_pos = subdf[y].max() + 0.01
		if (h1, h2) ==('stimulus', 'lights off'):
			#place higher and plot bar under
			y_pos += 0.18
			ax.plot([hue_to_offset[h1]+x_idx, hue_to_offset[h2]+x_idx], [y_pos, y_pos], color='k', lw=0.5,
           		clip_on=False)
			y_pos += 0.02
		if stars == 'n.s.':
			fontsize_=stars_fontsize*0.8
			# give color of gray
			color_='gray'
		else:
			fontsize_=stars_fontsize
			y_pos -= 0.05
			color_='k'
		ax.text(x_center, y_pos, stars, ha='center', va='bottom', fontsize=fontsize_, color=color_)
legend = ax.legend(loc=(0.82,0.6), fontsize=6*0.7)
legend.get_frame().set_linewidth(0.2)
ax.set_ylim(top=1)
import copy


if save_figs is True:
    plt.savefig(fig_dir +f'fig6_monkey_L_subsampled_to_A_resp_grayscreen_RS_corrs_{area}.pdf',transparent=True,bbox_inches='tight')
plt.show()

In [ ]:
# Fig Supp 8F top
random.seed(17)
df_monkey = df_monkey_all_seeds[df_monkey_all_seeds['subsample_seed']==random.choice(subsample_seeds)]
df_monkey_both=pd.merge(df_monkey[df_monkey['Activity Type']=='stimulus'],df_monkey[df_monkey['Activity Type']=='gray screen'].rename(columns={'EV':'EV gray screen','max r² val':'max r² val gray screen'}), on=['Date', 'Area',
    'Direction', 'SNR', 'split-half r', '1-vs-rest r²','control_shuffle'])
df_monkey_both_sorted = df_monkey_both.sort_values('EV gray screen')

# Calculate global min and max for consistent normalization
global_min = df_monkey_both_sorted['EV gray screen'].min()
global_max = df_monkey_both_sorted['EV gray screen'].max()

neuron_properties = ['max r² val', 'split-half r','1-vs-rest r²']
area='V4'
figsize=(2.5,0.75)
hspace=0.3
wspace=0.15
fig, axes = plt.subplots(1, 3, figsize=figsize)
for a, (ax, neuron_property) in  enumerate(zip(axes.flatten(), neuron_properties)):
    r_val = df_monkey_corr[(~df_monkey_corr['relationship'].isin(['EV_EV']))&(df_monkey_corr['Area']==area)&(df_monkey_corr['Value_Type']==neuron_property)&(df_monkey_corr['Activity Type']=='stimulus')]['corr'].mean()
    corr = plot_date(df_monkey_both_sorted,area, neuron_property, ax, r_loc=(0.05,0.85),s=8,
                    alpha=0.8, hue='EV gray screen', palette=v4_cmap, 
                    hue_norm=(global_min,global_max), linewidth=0.07, r_val=r_val)
    ax.set_ylim(top=1)
    ax.set_yticks([0,0.5,1])
    if a==0 or a==3:
        ax.set_ylabel('EV fraction', fontsize=6, labelpad=1)
    else: 
        ax.set_yticklabels('')
        ax.set(ylabel='')
    if a!=2:
        ax.legend_.remove()
    else:
        legend = ax.legend(loc=(1.1,0),fontsize=6*0.8)
        legend.set_title(title='EV gray screen',prop={'size':6*0.8})
        legend.get_frame().set_linewidth(0.2)
        ax.legend_.remove()
    ax.set_xlabel(None)
axes[0].set_xticks(ticks=[0,0.5])


plt.subplots_adjust(hspace=hspace, wspace=wspace)
# Create a colorbar with the custom colormap
norm = Normalize(global_min,global_max)
sm = plt.cm.ScalarMappable(cmap=v4_cmap, norm=norm)
sm.set_array([])  # Required for matplotlib to recognize the mappable object
cbar = plt.colorbar(sm, ax=ax, pad=0.02, shrink=0.9)
cbar.set_label('EV gray\nscreen', fontsize=4,y=1.25,ha='right', rotation=0)  # Customize label as needed
cbar.ax.tick_params(labelsize=4, pad=0.02)
cbar.ax.spines[:].set_linewidth(0.5)

# Adjust the colorbar's vertical position
# Get the original position of the colorbar
pos = cbar.ax.get_position()
cbar.ax.set_position([pos.x0, pos.y0 - 0.04, pos.width, pos.height])

if save_figs is True:
    plt.savefig(fig_dir + f'fig6_monkey_L_subsampled_to_A_resp_grayscreen_corrs_V4.pdf',transparent=True,bbox_inches='tight', dpi=4000)

In [ ]:

with open(os.path.join(project_root, 'results/fig_5', 'subsample_seeds.json'), 'r') as f:
    subsample_seeds = json.load(f)
    
main_monkey_name = 'L'
subsample_monkey_name = 'A'


stimulus_df_monkey_L_A = pd.DataFrame()
stimulus_df_monkey_L_A_null = pd.DataFrame()
pval_dict_all_seeds = {}
fontsize=7
df_monkey_corr_L_A_all_seeds = pd.DataFrame()



x= 'Value_Type'
y = 'corr'
hue = 'Activity Type'
area = 'V1'
plt.rc("hatch", linewidth=0.5)
hatch_size=5
xtick_labels = ['max r²', '1-vs-rest r²','SNR', 'split-half r',r'$\text{var}_{\text{time}}$',r'$\text{var}_{\text{repeats}}$',r'$\text{var}_{\text{stim}}$', ]


for seed in subsample_seeds:
	subsampled_monkey_stats_path = os.path.join(project_root,f'results/fig_5/monkey_L_subsampled_to_{subsample_monkey_name}',f'monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_seed{seed}_stats.pkl')
	with open(subsampled_monkey_stats_path, 'rb') as f:
		subsampled_monkey_stats = pickle.load(f)

	all_datataset_types_ = [k for k in subsampled_monkey_stats.keys() if k not in ['meta','monkey_alphas','monkey_alphas_glm','monkey_directionality_alphas']]
	df_monkey_ = make_monkey_df_neuron_properties(subsampled_monkey_stats, dataset_types=all_datataset_types_)
	df_monkey = df_monkey_[df_monkey_.control_shuffle==False]
	df_monkey_corr=make_corr_df_monkey(subsampled_monkey_stats, df_monkey)
	df_monkey_corr['subsample_seed'] = seed
	df_monkey_corr_L_A_all_seeds = pd.concat([df_monkey_corr_L_A_all_seeds, df_monkey_corr])
	
	order= [e for e in df_monkey_corr.Value_Type.unique() if e!='EV']
	filter_bool = df_monkey_corr['relationship'].isin(['EV_EV'])
	area_bool = df_monkey_corr['Area']==area
	mask = (~filter_bool)&area_bool
	hue_order = df_monkey_corr[mask][hue].unique()
	df_for_stats = df_monkey_corr[mask]
	res = pairwise_within_x_using_your_tests(
		df_for_stats, x=x, hue=hue, y=y,
		x_order=order, hue_order=hue_order,
		compare='ref',          # each bin vs the first bin (5 ms)
		mc_method='holm',
		perm_t=True, perm_type='ind', hierarchical=False,
		central_tendency='median', num_permutations=10000,
		mouse_or_date='Date'
	)


	
	for x_idx, x_i in enumerate(order):
		if x_i not in pval_dict_all_seeds:
			pval_dict_all_seeds[x_i] = {}
		pairs_ = res[x_i]['pairs']
		for (h1, h2) in pairs_:
			if (x_i == 'Variance across stimuli') & (((h1, h2) ==('stimulus', 'lights off')) or (h1, h2) ==('gray screen', 'lights off')):
				continue
			if (h1, h2) not in pval_dict_all_seeds[x_i]:
				pval_dict_all_seeds[x_i][(h1, h2)] = []
			pval = res[x_i]['p_adj'][res[x_i]['pairs'].index((h1,h2))] if (h1,h2) in res[x_i]['pairs'] else (res[x_i]['p_adj'][res[x_i]['pairs'].index((h2,h1))] if (h2,h1) in res[x_i]['pairs'] else None)
			pval_dict_all_seeds[x_i][(h1, h2)].append(pval)

# take median over seeds for p_vals_L_A
p_vals_L_A_median = {}
for x_i in pval_dict_all_seeds:
	p_vals_L_A_median[x_i] = {}
	for (h1, h2) in pval_dict_all_seeds[x_i]:
		p_vals_L_A_median[x_i][(h1, h2)] = np.median(pval_dict_all_seeds[x_i][(h1, h2)])

df_corr_mean_L_A = df_monkey_corr_L_A_all_seeds.groupby(['Area', 'Value_Type', 'Ref_property','relationship', 'Activity Type','Date'])['corr'].mean().reset_index()


In [ ]:
# Fig Supp 8E bottom
x= 'Value_Type'
y = 'corr'
hue = 'Activity Type'
area = 'V1'
plt.rc("hatch", linewidth=0.5)
hatch_size=5

order = ['max r² val', '1-vs-rest r²', 'SNR', 'split-half r',
       'Variance w/in trial\nacross timepoints',
       'Variance w/in timepoint\nacross trials',
       'Variance across stimuli']
hue_order = ['stimulus','gray screen']
xtick_labels = ['max r²', '1-vs-rest r²','SNR', 'split-half r',r'$\text{var}_{\text{time}}$',r'$\text{var}_{\text{repeats}}$',r'$\text{var}_{\text{stim}}$', ]
fig, ax = plt.subplots(figsize=(4,0.65))
mask, _ = plot_corr_bars(df_corr_mean_L_A, ax, area=area, hatch_size=hatch_size, x=x, hue=hue, y=y, order=order, hue_order=hue_order)
ax.set_xticks(ax.get_xticks(), labels=xtick_labels, rotation=0, ha='center', fontsize=6, )
ax.set_ylabel('corr w/ EV fraction', fontsize=6, loc='center')




res = p_vals_L_A_median
centers = np.arange(len(order))
n_h = len(hue_order)
# approximate seaborn offsets for grouped violins
width = 0.9
step = width / n_h if n_h > 0 else 0
start = -width/2 + step/2
offsets = [start + k*step for k in range(n_h)]
hue_to_offset = {h: offsets[i] for i, h in enumerate(hue_order)}
stars_fontsize = 5
# plot stars
for x_idx, x_i in enumerate(order):
	pairs_ = res[x_i]
	for (h1, h2) in pairs_:
		if (x_i == 'Variance across stimuli') & (((h1, h2) ==('stimulus', 'lights off')) or (h1, h2) ==('gray screen', 'lights off')):
			continue
		pval = res[x_i][(h1,h2)] if (h1,h2) in res[x_i] else (res[x_i]['p_adj'][res[x_i].index((h2,h1))] if (h2,h1) in res[x_i]['pairs'] else None)
		stars = p_to_stars(pval)
		subdf = df_for_stats[(df_for_stats[x]==x_i)&(df_for_stats[hue].isin([h1,h2]))]
		x_center = x_idx + 0.5*(hue_to_offset[h1] + hue_to_offset[h2])
		y_pos = subdf[y].max() + 0.01
		if (h1, h2) ==('stimulus', 'lights off'):
			#place higher and plot bar under
			y_pos += 0.18
			ax.plot([hue_to_offset[h1]+x_idx, hue_to_offset[h2]+x_idx], [y_pos, y_pos], color='k', lw=0.5,
           		clip_on=False)
			y_pos += 0.02
		if stars == 'n.s.':
			fontsize_=stars_fontsize*0.8
			# give color of gray
			color_='gray'
		else:
			fontsize_=stars_fontsize
			y_pos -= 0.05
			color_='k'
		ax.text(x_center, y_pos, stars, ha='center', va='bottom', fontsize=fontsize_, color=color_)
legend = ax.legend(loc=(0.82,0.75), fontsize=6*0.7)
legend.get_frame().set_linewidth(0.2)
ax.set_ylim(top=1, bottom=None)
import copy
v4_mask = mask.copy()

if save_figs is True:
    plt.savefig(fig_dir +f'fig6_monkey_L_subsampled_to_A_resp_grayscreen_RS_corrs_{area}.pdf',transparent=True,bbox_inches='tight')
plt.show()

In [ ]:
# Fig Supp 8F bottom
area='V1'

fig, axes = plt.subplots(1, 3, figsize=figsize)
for a, (ax, neuron_property) in  enumerate(zip(axes.flatten(), neuron_properties)):
    r_val = df_monkey_corr[(~df_monkey_corr['relationship'].isin(['EV_EV']))&(df_monkey_corr['Area']==area)&(df_monkey_corr['Value_Type']==neuron_property)&(df_monkey_corr['Activity Type']=='stimulus')]['corr'].mean()
    corr = plot_date(df_monkey_both_sorted,area, neuron_property, ax, r_loc=(0.05,0.85),s=8,
                    alpha=0.8, hue='EV gray screen', palette=v1_cmap, 
                    r_val=r_val, linewidth=0.07, hue_norm=(global_min,global_max))
    ax.set_ylim(top=1)
    ax.set_yticks([0,0.5,1])
    if a==0 or a==3:
        ax.set_ylabel('EV fraction', fontsize=6, labelpad=1)
    else: 
        ax.set_yticklabels('')
        ax.set(ylabel='')
    ax.legend_.remove()
    
axes[0].set_xticks(ticks=[0,0.5])
plt.subplots_adjust(hspace=hspace, wspace=wspace)

# Create a colorbar with the custom colormap
norm = Normalize(global_min,global_max)
sm = plt.cm.ScalarMappable(cmap=v1_cmap, norm=norm)
sm.set_array([])  # Required for matplotlib to recognize the mappable object
cbar = plt.colorbar(sm, ax=ax, pad=0.02, shrink=0.9)
cbar.set_label('EV gray\nscreen', fontsize=4,y=1.25,ha='right', rotation=0)  # Customize label as needed
cbar.ax.tick_params(labelsize=4, pad=0.02)
cbar.ax.spines[:].set_linewidth(0.5)

# Adjust the colorbar's vertical position
# Get the original position of the colorbar
pos = cbar.ax.get_position()
cbar.ax.set_position([pos.x0, pos.y0 - 0.04, pos.width, pos.height])

if save_figs is True:
    plt.savefig(fig_dir + f'fig6_monkey_L_subsampled_to_A_resp_grayscreen_corrs_V1.pdf',transparent=True,bbox_inches='tight', dpi=4000)

## Monkey $L_{D}$ subsampling

In [ ]:
with open(os.path.join(project_root, 'results/fig_5', 'subsample_seeds.json'), 'r') as f:
    subsample_seeds = json.load(f)
    
main_monkey_name = 'L'
subsample_monkey_name = 'D'


stimulus_df_monkey_L_A = pd.DataFrame()
stimulus_df_monkey_L_A_null = pd.DataFrame()
pval_dict_all_seeds = {}
fontsize=7
df_monkey_all_seeds = pd.DataFrame()
df_monkey_corr_L_A_all_seeds = pd.DataFrame()



x= 'Value_Type'
y = 'corr'
hue = 'Activity Type'
area = 'V4'
plt.rc("hatch", linewidth=0.5)
hatch_size=5
xtick_labels = ['max r²', '1-vs-rest r²','SNR', 'split-half r',r'$\text{var}_{\text{time}}$',r'$\text{var}_{\text{repeats}}$',r'$\text{var}_{\text{stim}}$', ]

hue_order = ['stimulus', 'gray screen', 'lights off']
order = ['max r² val', '1-vs-rest r²', 'SNR', 'split-half r',
       'Variance w/in trial\nacross timepoints',
       'Variance w/in timepoint\nacross trials',
       'Variance across stimuli']
for seed in subsample_seeds:
	subsampled_monkey_stats_path = os.path.join(project_root,f'results/fig_5/monkey_L_subsampled_to_{subsample_monkey_name}',f'monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_seed{seed}_stats.pkl')
	with open(subsampled_monkey_stats_path, 'rb') as f:
		subsampled_monkey_stats = pickle.load(f)
	all_datataset_types_ = [k for k in subsampled_monkey_stats.keys() if k not in ['meta','monkey_alphas','monkey_alphas_glm','monkey_directionality_alphas']]
	df_monkey_ = make_monkey_df_neuron_properties(subsampled_monkey_stats, dataset_types=all_datataset_types_)
	df_monkey = df_monkey_[df_monkey_.control_shuffle==False]
	df_monkey['subsample_seed'] = seed
	df_monkey_all_seeds = pd.concat([df_monkey_all_seeds, df_monkey])
	df_monkey_corr=make_corr_df_monkey(subsampled_monkey_stats, df_monkey)
	df_monkey_corr['subsample_seed'] = seed
	df_monkey_corr_L_A_all_seeds = pd.concat([df_monkey_corr_L_A_all_seeds, df_monkey_corr])

	order= [e for e in df_monkey_corr.Value_Type.unique() if e!='EV']
	filter_bool = df_monkey_corr['relationship'].isin(['EV_EV'])
	area_bool = df_monkey_corr['Area']==area
	mask = (~filter_bool)&area_bool

	hue_order = df_monkey_corr[mask][hue].unique()
	df_for_stats = df_monkey_corr[mask]
	res = pairwise_within_x_using_your_tests(
		df_for_stats, x=x, hue=hue, y=y,
		x_order=order, hue_order=hue_order,
		compare='ref',          # each bin vs the first bin (5 ms)
		mc_method='holm',
		perm_t=True, perm_type='ind', hierarchical=False,
		central_tendency='median', num_permutations=10000,
		mouse_or_date='Date'
	)
	
	for x_idx, x_i in enumerate(order):
		if x_i not in pval_dict_all_seeds:
			pval_dict_all_seeds[x_i] = {}
		pairs_ = res[x_i]['pairs']
		for (h1, h2) in pairs_:
			if (x_i == 'Variance across stimuli') & (((h1, h2) ==('stimulus', 'lights off')) or (h1, h2) ==('gray screen', 'lights off')):
				continue
			if (h1, h2) not in pval_dict_all_seeds[x_i]:
				pval_dict_all_seeds[x_i][(h1, h2)] = []
			pval = res[x_i]['p_adj'][res[x_i]['pairs'].index((h1,h2))] if (h1,h2) in res[x_i]['pairs'] else (res[x_i]['p_adj'][res[x_i]['pairs'].index((h2,h1))] if (h2,h1) in res[x_i]['pairs'] else None)
			pval_dict_all_seeds[x_i][(h1, h2)].append(pval)

# take median over seeds for p_vals_L_A
p_vals_L_A_median = {}
for x_i in pval_dict_all_seeds:
	p_vals_L_A_median[x_i] = {}
	for (h1, h2) in pval_dict_all_seeds[x_i]:
		p_vals_L_A_median[x_i][(h1, h2)] = np.median(pval_dict_all_seeds[x_i][(h1, h2)])



In [ ]:
# Fig 9C top
x= 'Value_Type'
y = 'corr'
hue = 'Activity Type'
area = 'V4'
plt.rc("hatch", linewidth=0.5)
hatch_size=5

order = ['max r² val', '1-vs-rest r²', 'SNR', 'split-half r',
       'Variance w/in trial\nacross timepoints',
       'Variance w/in timepoint\nacross trials',
       'Variance across stimuli']
hue_order = ['stimulus', 'gray screen', 'lights off']
xtick_labels = ['max r²', '1-vs-rest r²','SNR', 'split-half r',r'$\text{var}_{\text{time}}$',r'$\text{var}_{\text{repeats}}$',r'$\text{var}_{\text{stim}}$', ]
fig, ax = plt.subplots(figsize=(3.8,0.65))
df_corr_mean_L_A = df_monkey_corr_L_A_all_seeds.groupby(['Area', 'Value_Type', 'Ref_property','relationship', 'Activity Type','Dataset Type','Date'])['corr'].mean().reset_index()
df_corr_mean_L_A = df_corr_mean_L_A.sort_values(by='Activity Type', key=lambda x: x.map({'stimulus':0, 'gray screen':1, 'lights off':2}))
mask, _ = plot_corr_bars(df_corr_mean_L_A, ax, area=area, hatch_size=hatch_size, x=x, hue=hue, 
                        y=y, order=order, hue_order=hue_order)
ax.set_xticks(ax.get_xticks(), labels=xtick_labels, rotation=0, ha='center', fontsize=6, )
ax.set_ylabel('corr w/ EV fraction', fontsize=6, loc='center')
df_for_stats = df_corr_mean_L_A[mask]

res = p_vals_L_A_median
centers = np.arange(len(order))
n_h = len(hue_order)
# approximate seaborn offsets for grouped violins
width = 0.9
step = width / n_h if n_h > 0 else 0
start = -width/2 + step/2
offsets = [start + k*step for k in range(n_h)]
hue_to_offset = {h: offsets[i] for i, h in enumerate(hue_order)}
stars_fontsize = 5
# plot stars
for x_idx, x_i in enumerate(order):
	pairs_ = res[x_i]
	for (h1, h2) in pairs_:
		if (x_i == 'Variance across stimuli') & (((h1, h2) ==('stimulus', 'lights off')) or (h1, h2) ==('gray screen', 'lights off')):
			continue
		pval = res[x_i][(h1,h2)] if (h1,h2) in res[x_i] else (res[x_i]['p_adj'][res[x_i].index((h2,h1))] if (h2,h1) in res[x_i]['pairs'] else None)
		stars = p_to_stars(pval)
		subdf = df_for_stats[(df_for_stats[x]==x_i)&(df_for_stats[hue].isin([h1,h2]))]
		x_center = x_idx + 0.5*(hue_to_offset[h1] + hue_to_offset[h2])
		y_pos = subdf[y].max()*1.03
		if (h1, h2) ==('stimulus', 'lights off'):
			#place higher and plot bar under
			y_pos += 0.18
			ax.plot([hue_to_offset[h1]+x_idx, hue_to_offset[h2]+x_idx], [y_pos, y_pos], color='k', lw=0.5,
           		clip_on=False)
			y_pos += 0.02
		if stars == 'n.s.':
			fontsize_=stars_fontsize*0.8
			# give color of gray
			color_='gray'
		else:
			fontsize_=stars_fontsize
			y_pos -= 0.05
			color_='k'
		ax.text(x_center, y_pos, stars, ha='center', va='bottom', fontsize=fontsize_, color=color_)
legend = ax.legend(loc=(0.82,0.6), fontsize=6*0.7)
legend.get_frame().set_linewidth(0.2)
ax.set_ylim(top=1, bottom=None)


if save_figs is True:
    plt.savefig(fig_dir +f'fig6_monkey_L_subsampled_to_D_resp_grayscreen_RS_corrs_{area}.pdf',transparent=True,bbox_inches='tight')
plt.show()

In [ ]:
# Fig Supp 9D top

random.seed(17)
df_monkey = df_monkey_all_seeds[df_monkey_all_seeds['subsample_seed']==random.choice(subsample_seeds)]
df_monkey_both=pd.merge(df_monkey[df_monkey['Activity Type']=='stimulus'],df_monkey[df_monkey['Activity Type']=='gray screen'].rename(columns={'EV':'EV gray screen','max r² val':'max r² val gray screen'}), on=['Date', 'Area',
    'Direction', 'SNR', 'split-half r', '1-vs-rest r²','control_shuffle'])
df_monkey_both_sorted = df_monkey_both.sort_values('EV gray screen')

# Calculate global min and max for consistent normalization
global_min = df_monkey_both_sorted['EV gray screen'].min()
global_max = df_monkey_both_sorted['EV gray screen'].max()

neuron_properties = ['max r² val', 'split-half r','1-vs-rest r²']
area='V4'
figsize=(2.5,0.75)
hspace=0.3
wspace=0.15
fig, axes = plt.subplots(1, 3, figsize=figsize)
for a, (ax, neuron_property) in  enumerate(zip(axes.flatten(), neuron_properties)):
	r_val = df_monkey_corr[(~df_monkey_corr['relationship'].isin(['EV_EV']))&(df_monkey_corr['Area']==area)&(df_monkey_corr['Value_Type']==neuron_property)&(df_monkey_corr['Activity Type']=='stimulus')]['corr'].mean()
	corr = plot_date(df_monkey_both_sorted,area, neuron_property, ax, r_loc=(0.05,0.85),s=12,
					alpha=0.8, hue='EV gray screen', palette=v4_cmap, 
					hue_norm=(global_min,global_max), linewidth=0.07, r_val=r_val)
	ax.set_ylim(top=0.75, bottom=None)
	ax.set_yticks([0,0.5])
	if a==0 or a==3:
		ax.set_ylabel('EV fraction', fontsize=6, labelpad=1)
	else: 
		ax.set_yticklabels('')
		ax.set(ylabel='')
	if a!=2:
		ax.legend_.remove()
	else:
		legend = ax.legend(loc=(1.1,0),fontsize=6*0.8)
		legend.set_title(title='EV gray screen',prop={'size':6*0.8})
		legend.get_frame().set_linewidth(0.2)
		ax.legend_.remove()
	ax.set_xlabel(None)
	# ax.tick_params(axis='x', which='both', labelsize=5.5)

axes[0].set_xticks(ticks=[0,0.5])
axes[1].set_xticks(ticks=[0,0.5])	

plt.subplots_adjust(hspace=hspace, wspace=wspace)
# Create a colorbar with the custom colormap
norm = Normalize(global_min,global_max)
sm = plt.cm.ScalarMappable(cmap=v4_cmap, norm=norm)
sm.set_array([])  # Required for matplotlib to recognize the mappable object
cbar = plt.colorbar(sm, ax=ax, pad=0.02, shrink=0.9)
cbar.set_label('EV gray\nscreen', fontsize=4,y=1.25,ha='right', rotation=0)  # Customize label as needed
cbar.ax.tick_params(labelsize=4, pad=0.02)
cbar.ax.spines[:].set_linewidth(0.5)

# Adjust the colorbar's vertical position
# Get the original position of the colorbar
pos = cbar.ax.get_position()
cbar.ax.set_position([pos.x0, pos.y0 - 0.04, pos.width, pos.height])
plt.subplots_adjust(wspace=0.2)
if save_figs is True:
    plt.savefig(fig_dir + f'fig6_monkey_L_subsampled_to_D_resp_grayscreen_corrs_V4.pdf',transparent=True,bbox_inches='tight', dpi=4000)

In [ ]:
with open(os.path.join(project_root, 'results/fig_4', 'subsample_seeds.json'), 'r') as f:
    subsample_seeds = json.load(f)
    
main_monkey_name = 'L'
subsample_monkey_name = 'D'


stimulus_df_monkey_L_A = pd.DataFrame()
stimulus_df_monkey_L_A_null = pd.DataFrame()
pval_dict_all_seeds = {}
fontsize=7
df_monkey_corr_L_A_all_seeds = pd.DataFrame()



x= 'Value_Type'
y = 'corr'
hue = 'Activity Type'
area = 'V1'
plt.rc("hatch", linewidth=0.5)
hatch_size=5
xtick_labels = ['max r²', '1-vs-rest r²','SNR', 'split-half r',r'$\text{var}_{\text{time}}$',r'$\text{var}_{\text{repeats}}$',r'$\text{var}_{\text{stim}}$', ]


for seed in subsample_seeds:
	subsampled_monkey_stats_path = os.path.join(project_root,f'results/fig_5/monkey_L_subsampled_to_{subsample_monkey_name}',f'monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_seed{seed}_stats.pkl')
	with open(subsampled_monkey_stats_path, 'rb') as f:
		subsampled_monkey_stats = pickle.load(f)

	all_datataset_types_ = [k for k in subsampled_monkey_stats.keys() if k not in ['meta','monkey_alphas','monkey_alphas_glm','monkey_directionality_alphas']]
	df_monkey_ = make_monkey_df_neuron_properties(subsampled_monkey_stats, dataset_types=all_datataset_types_)
	df_monkey = df_monkey_[df_monkey_.control_shuffle==False]
	df_monkey_corr=make_corr_df_monkey(subsampled_monkey_stats, df_monkey)
	df_monkey_corr['subsample_seed'] = seed
	df_monkey_corr_L_A_all_seeds = pd.concat([df_monkey_corr_L_A_all_seeds, df_monkey_corr])
	
	order= [e for e in df_monkey_corr.Value_Type.unique() if e!='EV']
	filter_bool = df_monkey_corr['relationship'].isin(['EV_EV'])
	area_bool = df_monkey_corr['Area']==area
	mask = (~filter_bool)&area_bool

	hue_order = df_monkey_corr[mask][hue].unique()
	df_for_stats = df_monkey_corr[mask]
	res = pairwise_within_x_using_your_tests(
		df_for_stats, x=x, hue=hue, y=y,
		x_order=order, hue_order=hue_order,
		compare='ref',          # each bin vs the first bin (5 ms)
		mc_method='holm',
		perm_t=True, perm_type='ind', hierarchical=False,
		central_tendency='median', num_permutations=10000,
		mouse_or_date='Date'
	)
	
	for x_idx, x_i in enumerate(order):
		if x_i not in pval_dict_all_seeds:
			pval_dict_all_seeds[x_i] = {}
		pairs_ = res[x_i]['pairs']
		for (h1, h2) in pairs_:
			if (x_i == 'Variance across stimuli') & (((h1, h2) ==('stimulus', 'lights off')) or (h1, h2) ==('gray screen', 'lights off')):
				continue
			if (h1, h2) not in pval_dict_all_seeds[x_i]:
				pval_dict_all_seeds[x_i][(h1, h2)] = []
			pval = res[x_i]['p_adj'][res[x_i]['pairs'].index((h1,h2))] if (h1,h2) in res[x_i]['pairs'] else (res[x_i]['p_adj'][res[x_i]['pairs'].index((h2,h1))] if (h2,h1) in res[x_i]['pairs'] else None)
			pval_dict_all_seeds[x_i][(h1, h2)].append(pval)

# take median over seeds for p_vals_L_A
p_vals_L_A_median = {}
for x_i in pval_dict_all_seeds:
	p_vals_L_A_median[x_i] = {}
	for (h1, h2) in pval_dict_all_seeds[x_i]:
		p_vals_L_A_median[x_i][(h1, h2)] = np.median(pval_dict_all_seeds[x_i][(h1, h2)])
df_corr_mean_L_A = df_monkey_corr_L_A_all_seeds.groupby(['Area', 'Value_Type', 'Ref_property','relationship', 'Activity Type','Date'])['corr'].median().reset_index()



In [ ]:
import random
random.seed(17)
df_monkey = df_monkey_all_seeds[df_monkey_all_seeds['subsample_seed']==random.choice(subsample_seeds)]
df_monkey_both=pd.merge(df_monkey[df_monkey['Activity Type']=='stimulus'],df_monkey[df_monkey['Activity Type']=='gray screen'].rename(columns={'EV':'EV gray screen','max r² val':'max r² val gray screen'}), on=['Date', 'Area',
    'Direction', 'SNR', 'split-half r', '1-vs-rest r²','control_shuffle'])
df_monkey_both_sorted = df_monkey_both.sort_values('EV gray screen')

# Calculate global min and max for consistent normalization
global_min = df_monkey_both_sorted['EV gray screen'].min()
global_max = df_monkey_both_sorted['EV gray screen'].max()

neuron_properties = ['max r² val', 'split-half r','1-vs-rest r²']

In [ ]:
# Fig 9C bottom
x= 'Value_Type'
y = 'corr'
hue = 'Activity Type'
area = 'V1'
plt.rc("hatch", linewidth=0.5)
hatch_size=5

order = ['max r² val', '1-vs-rest r²', 'SNR', 'split-half r',
       'Variance w/in trial\nacross timepoints',
       'Variance w/in timepoint\nacross trials',
       'Variance across stimuli']
hue_order = ['stimulus', 'gray screen', 'lights off']
xtick_labels = ['max r²', '1-vs-rest r²','SNR', 'split-half r',r'$\text{var}_{\text{time}}$',r'$\text{var}_{\text{repeats}}$',r'$\text{var}_{\text{stim}}$', ]

fig, ax = plt.subplots(figsize=(3.8,0.65))
df_corr_mean_L_A = df_monkey_corr_L_A_all_seeds.groupby(['Area', 'Value_Type', 'Ref_property','relationship', 'Activity Type','Dataset Type','Date'])['corr'].mean().reset_index()
df_corr_mean_L_A = df_corr_mean_L_A.sort_values(by='Activity Type', key=lambda x: x.map({'stimulus':0, 'gray screen':1, 'lights off':2}))
mask, _ = plot_corr_bars(df_corr_mean_L_A, ax, area=area, hatch_size=hatch_size, x=x, hue=hue, 
                        y=y, order=order, hue_order=hue_order)
ax.set_xticks(ax.get_xticks(), labels=xtick_labels, rotation=0, ha='center', fontsize=6, )
ax.set_ylabel('corr w/ EV fraction', fontsize=6, loc='center')

res = p_vals_L_A_median
centers = np.arange(len(order))
n_h = len(hue_order)
# approximate seaborn offsets for grouped violins
width = 0.9
step = width / n_h if n_h > 0 else 0
start = -width/2 + step/2
offsets = [start + k*step for k in range(n_h)]
hue_to_offset = {h: offsets[i] for i, h in enumerate(hue_order)}
stars_fontsize = 5
# plot stars
df_for_stats = df_corr_mean_L_A[mask]
for x_idx, x_i in enumerate(order):
	pairs_ = res[x_i]
	for (h1, h2) in pairs_:
		if (x_i == 'Variance across stimuli') & (((h1, h2) ==('stimulus', 'lights off')) or (h1, h2) ==('gray screen', 'lights off')):
			continue
		pval = res[x_i][(h1,h2)] if (h1,h2) in res[x_i] else (res[x_i]['p_adj'][res[x_i].index((h2,h1))] if (h2,h1) in res[x_i]['pairs'] else None)
		stars = p_to_stars(pval)
		subdf = df_for_stats[(df_for_stats[x]==x_i)&(df_for_stats[hue].isin([h1,h2]))]
		x_center = x_idx + 0.5*(hue_to_offset[h1] + hue_to_offset[h2])
		y_pos = subdf[y].max()*1.03
		if (h1, h2) ==('stimulus', 'lights off'):
			#place higher and plot bar under
			y_pos += 0.18
			# if x_i =='Variance w/in trial\nacross timepoints':
			# 	y_pos += 0.09
			ax.plot([hue_to_offset[h1]+x_idx, hue_to_offset[h2]+x_idx], [y_pos, y_pos], color='k', lw=0.5,
           		clip_on=False)
			y_pos += 0.02
		if stars == 'n.s.':
			fontsize_=stars_fontsize*0.8
			# give color of gray
			color_='gray'
		else:
			fontsize_=stars_fontsize
			y_pos -= 0.05
			color_='k'
		ax.text(x_center, y_pos, stars, ha='center', va='bottom', fontsize=fontsize_, color=color_)
legend = ax.legend(loc=(0.82,0.85), fontsize=6*0.7)
legend.get_frame().set_linewidth(0.2)
ax.set_ylim(top=1, bottom=None)

if save_figs is True:
    plt.savefig(fig_dir +f'fig6_monkey_L_subsampled_to_D_resp_grayscreen_RS_corrs_{area}.pdf',transparent=True,bbox_inches='tight')
plt.show()

In [ ]:
# Fig Supp 9D bottom
area='V1'
figsize=(2.5,0.75)
fig, axes = plt.subplots(1, 3, figsize=figsize)
for a, (ax, neuron_property) in  enumerate(zip(axes.flatten(), neuron_properties)):
    r_val = df_monkey_corr[(~df_monkey_corr['relationship'].isin(['EV_EV']))&(df_monkey_corr['Area']==area)&(df_monkey_corr['Value_Type']==neuron_property)&(df_monkey_corr['Activity Type']=='stimulus')]['corr'].mean()
    corr = plot_date(df_monkey_both_sorted,area, neuron_property, ax, r_loc=(0.05,0.85),s=12,
                    alpha=0.8, hue='EV gray screen', palette=v1_cmap, 
                    r_val=r_val, linewidth=0.07, hue_norm=(global_min,global_max))
    ax.set_ylim(top=0.75, bottom=None)
    ax.set_yticks([0,0.5])
    if a==0 or a==3:
        ax.set_ylabel('EV fraction', fontsize=6, labelpad=1)
    else: 
        ax.set_yticklabels('')
        ax.set(ylabel='')
    ax.legend_.remove()
hspace=0.3
wspace=0.15  
axes[0].set_xticks(ticks=[0,0.5])
axes[1].set_xticks(ticks=[0,0.5])
axes[2].set_xticks(ticks=[0,0.5])
plt.subplots_adjust(hspace=hspace, wspace=wspace)

# Create a colorbar with the custom colormap
norm = Normalize(global_min,global_max)
sm = plt.cm.ScalarMappable(cmap=v1_cmap, norm=norm)
sm.set_array([])  # Required for matplotlib to recognize the mappable object
cbar = plt.colorbar(sm, ax=ax, pad=0.02, shrink=0.9)
cbar.set_label('EV gray\nscreen', fontsize=4,y=1.25,ha='right', rotation=0)  # Customize label as needed
cbar.ax.tick_params(labelsize=4, pad=0.02)
cbar.ax.spines[:].set_linewidth(0.5)

# Adjust the colorbar's vertical position
# Get the original position of the colorbar
pos = cbar.ax.get_position()
cbar.ax.set_position([pos.x0, pos.y0 - 0.04, pos.width, pos.height])
plt.subplots_adjust(wspace=0.2)
if save_figs is True:
    plt.savefig(fig_dir + f'fig6_monkey_L_subsampled_to_D_resp_grayscreen_corrs_V1.pdf',transparent=True,bbox_inches='tight', dpi=4000)